In [1]:
# Prior libraries
import pandas as pd
import numpy as np
import os
import string

# To Get Combinatiobs
import itertools

# Datetime Libraries
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import time

# Trend Seasonality
from statsmodels.tsa.seasonal import STL

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# SAS Connection Library
import swat

# In Order To Read Config File
import json

# Model Preprocess Librarires
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Machine Learning Algorithm Libraries
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Other Libraries
import math
import logging
from tqdm import tqdm
import cx_Oracle 
from sqlalchemy import create_engine

import multiprocess as mp
from functools import partial

# Database Libraries
import sqlite3
from sqlite3 import Error

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max.columns", 100)
pd.set_option("display.max.rows", 100)

C:\Users\tunahan.aktas\Anaconda3\lib\site-packages\mpl_toolkits\mplot3d\__init__.py:1: MatplotlibDeprecationWarning: 
The deprecated function was deprecated in Matplotlib 3.4 and will be removed two minor releases later.
  from .axes3d import Axes3D


In [2]:
f = open("../config.json", "r")
params_ = json.loads(f.read())

In [3]:
data_name_dict = {
    
    1: "ocak",
    2: "subat",
    3: "mart",
    4: "nisan",
    5: "mayis",
    6: "haziran",
    7: "temmuz",
    8: "agustos",
    9: "eylul",
    10: "ekim",
    11: "kasim",
    12: "aralik"
}

---

# DB Functions

In [4]:
def create_connection(db_file):
    """ Create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [5]:
def create_new_table(df_name, database_name, table_name):
    """
    Create new table into given database.
    
    Parameters
    ----------
    df_name : DataFrame
        The DataFrame in order to create as a new table in the database.
    
    database_name : str
        Specify the name of the Database that you want to create a new table from a DataFrame.
    
    table_name : str
        Specify the name of the table that will be created into the database.
        
    """
    access_ = True
    os.chdir("../data/db/")
    database_name += ".sqlite"
    while access_:
        if database_name in os.listdir():
            access_ = False
            try:
                conn = sqlite3.connect(database_name)
                df_name.to_sql(table_name, conn, index=False)
                print(f"Table '{table_name}' is created into '{database_name}' database.")
            except ValueError as e:
                print(e)
        else:
            print("The provided database name is incorrect. Please check again 'database_name' parameter.")
            break
    conn.close()
    os.chdir("../../"+"data prep/")

In [6]:
def upload_new_results(new_results, database_name, table_name):
    """
    Update a table in given database.
    
    Parameters
    ----------
    new_results : DataFrame
        The DataFrame in order to upload new results into given database.
    
    database_name : str
        Specify the name of the Database that you want to create a new table from a DataFrame.
    
    table_name : str
        Specify the name of the table that will be created into the database.
        
    """
    os.chdir("../data/db/")
    conn = sqlite3.connect(database_name + ".sqlite")
    cursor = conn.cursor()
    sql_query = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(sql_query)
    print("Previous results are dropped.")
    os.chdir("../../"+"data prep/")
    conn.close()
    create_new_table(new_results, database_name, table_name)
    print("New results are uploaded.")

In [7]:
def update_table(dataframe_to_append, database_name, table_name):
    """
    Update a table in given database.
    
    Parameters
    ----------
    dataframe_to_append : DataFrame
        The DataFrame in order to upload new results into backup table which contains previous predictions.
    
    database_name : str
        Specify the name of the Database that you want to create a new table from a DataFrame.
    
    table_name : str
        Specify the name of the table that will be created into the database.
        
    """
    os.chdir("../data/db/")
    conn = sqlite3.connect(database_name + ".sqlite")
    dataframe_to_append.to_sql(table_name, con=conn, if_exists='append', index=False)
    print(f"New data is inserted into {table_name} table.")
    os.chdir("../../"+"data prep/")
    conn.close()

# Read Files

In [8]:
start_all_process = datetime.now()

In [9]:
start_time = datetime.now()

In [10]:
# List of Excel Files
pas_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["pasifik_satis_path"]) if doc_.startswith("Siparişe_göre_Sales_History")])
hor_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["horizon_satis_path"]) if doc_.startswith("Horizon_Saha_")])
btt_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["btt_satis_path"]) if doc_.startswith("Demand Sensing Sales History") or doc_.startswith("Demand_Sensing_Sales_")])

saha_aktivite_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["horizon_aktivite_path"]) if doc_.startswith("Demand_Sensing_Saha_Aktivit") or doc_.startswith("Demand Sensing Saha Aktivit")])
pasifik_aktivite_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["pasifik_aktivite_path"]) if doc_.startswith("Pasifik Aktivite Datası")])

portfoy_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["portfoy_path"]) if doc_.startswith("Portföy")])
eslenik_kod_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["eslenik_kod_path"]) if doc_.startswith("Ürün Eşlenik kodlar")])
kapsam_listeli = sorted([doc_ for doc_ in os.listdir(params_["path"]["kapsam_path"]) if doc_.startswith("Listeli Ürün")])
pas_siparis_lst = sorted([doc_ for doc_ in os.listdir(params_["path"]["pasifik_siparis_path"]) if doc_.startswith("Siparişe_göre_Sales_History")])

In [11]:
end_time = datetime.now()
print('Dosya Listelerini Okuma İşlemi: {}'.format(end_time - start_time))

Dosya Listelerini Okuma İşlemi: 0:00:00.036999


---
# Read Data
---

In [12]:
chng_cols_beginning = {'Year': 'Yıl', 'Quarter': 'Çeyrek', 'Month': 'Ay', 
                       'Company Code': 'Şirket Kodu', 'Main Category Name': 'Ana Kategori Adı', 
                       'Category Name': 'Kategori Adı', 'Brand Name': 'Marka Adı', 'Product Code': 'Ürün Kodu', 
                       'Product Name': 'Ürün Adı', "Ürün Adı (Mobis)": 'Ürün Adı'}

In [13]:
def read_pas(paths_pas, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_pas+loop_list, sheet_name="Ürün Bazlı", usecols="B:O").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [14]:
def read_hor(paths_hor, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_hor+loop_list, skiprows=1, sheet_name="Horizon Saha Satış", usecols="B:L").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [15]:
def read_btt(paths_btt, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_btt+loop_list, skiprows=1, sheet_name="BTT SAP Satış", usecols="B:N").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [16]:
start_time = datetime.now()

In [17]:
pasifik_df_all = []
horizon_saha_df_all = []
btt_df_all = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    
    paths_pas = params_["path"]["pasifik_satis_path"]
    paths_hor = params_["path"]["horizon_satis_path"]
    paths_btt = params_["path"]["btt_satis_path"]

    new_col_names = chng_cols_beginning

    loop_pas = pas_lst
    loop_hor = hor_lst
    loop_btt = btt_lst
    
    func_pas = partial(read_pas, paths_pas, new_col_names)
    func_hor = partial(read_hor, paths_hor, new_col_names)
    func_btt = partial(read_btt, paths_btt, new_col_names)
    
    with mp.Pool(available_cpu) as p:
        pasifik_df_all.append(p.map(func_pas, loop_pas))
    with mp.Pool(available_cpu) as p:
        horizon_saha_df_all.append(p.map(func_hor, loop_hor))
    with mp.Pool(available_cpu) as p:
        btt_df_all.append(p.map(func_btt, loop_btt))

In [18]:
end_time = datetime.now()
print('Dataların yüklenmesi: {}'.format(end_time - start_time))

Dataların yüklenmesi: 0:02:25.825284


In [19]:
pasifik_df_all = pd.concat(pasifik_df_all[0], ignore_index=True)
horizon_saha_df_all = pd.concat(horizon_saha_df_all[0], ignore_index=True)
btt_df_all = pd.concat(btt_df_all[0], ignore_index=True)

---
# Read Data Koli İçi Adet
---

In [20]:
chng_cols_beginning = {'Year': 'Yıl', 'Quarter': 'Çeyrek', 'Month': 'Ay', 
                       'Company Code': 'Şirket Kodu', 'Main Category Name': 'Ana Kategori Adı', 
                       'Category Name': 'Kategori Adı', 'Brand Name': 'Marka Adı', 'Product Code': 'Ürün Kodu', 
                       'Product Name': 'Ürün Adı', "Ürün Adı (Mobis)": 'Ürün Adı'}

In [21]:
def read_pas_koli_ici_adet(paths_pas, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_pas+loop_list, sheet_name="Koli içi adet", skiprows=1, usecols="C:D").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [22]:
def read_hor_koli_ici_adet(paths_btt, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_btt+loop_list, skiprows=1, sheet_name="Horizon Saha Satış", usecols="B,D,E,I,K").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [23]:
def read_btt_koli_ici_adet(paths_btt, new_col_names, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_btt+loop_list, skiprows=1, sheet_name="BTT SAP Satış", usecols="B,D,I,K").rename(columns=new_col_names)
    print("Okuma bitti ---->", loop_list)
    return new_df

In [24]:
start_time = datetime.now()

In [25]:
pasifik_koli_ici_adet = []
horizon_koli_ici_adet = []
btt_koli_ici_adet = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    
    paths_pas = params_["path"]["pasifik_satis_path"]
    paths_btt = params_["path"]["btt_satis_path"]

    new_col_names = chng_cols_beginning

    loop_pas = [loop_pas[-1]]
    loop_btt = btt_lst
    
    func_pas = partial(read_pas_koli_ici_adet, paths_pas, new_col_names)
    func_hor = partial(read_hor_koli_ici_adet, paths_btt, new_col_names)
    func_btt = partial(read_btt_koli_ici_adet, paths_btt, new_col_names)
    
    with mp.Pool(available_cpu) as p:
        pasifik_koli_ici_adet.append(p.map(func_pas, loop_pas))
    with mp.Pool(available_cpu) as p:
        horizon_koli_ici_adet.append(p.map(func_hor, loop_btt))
    with mp.Pool(available_cpu) as p:
        btt_koli_ici_adet.append(p.map(func_btt, loop_btt))

In [26]:
end_time = datetime.now()
print('Dataların yüklenmesi: {}'.format(end_time - start_time))

Dataların yüklenmesi: 0:01:09.296317


In [27]:
lower_case = list(string.ascii_lowercase)
upper_case = list(string.ascii_uppercase)
lower_upper_case = upper_case + lower_case

In [28]:
pasifik_koli_ici_adet = pd.concat(pasifik_koli_ici_adet[0])
horizon_koli_ici_adet = pd.concat(horizon_koli_ici_adet[0])
btt_koli_ici_adet = pd.concat(btt_koli_ici_adet[0])

In [29]:
pasifik_koli_ici_adet_backup = pasifik_koli_ici_adet.copy()

In [30]:
ltrs = list(string.ascii_letters) # Alfabede bulunan tüm harfleri tutan liste. Bunu, koli içi adet dataframe'deki harf içeren ürün kodlarını elemek için tutuyoruz.

In [31]:
pasifik_koli_ici_adet.rename(columns={"Ürün Kodu.1": "en_guncel_kod", "Koli İçi Adet": "koli_ici_adet"}, inplace=True)
pasifik_koli_ici_adet = pasifik_koli_ici_adet[~(pasifik_koli_ici_adet["en_guncel_kod"].str.contains("|".join(ltrs), regex=True))].reset_index(drop=True)

In [32]:
pasifik_koli_ici_adet.en_guncel_kod = pasifik_koli_ici_adet.en_guncel_kod.apply(lambda x: int(x.replace(" ", "")))

In [33]:
horizon_koli_ici_adet.rename(columns={"Yıl": "yil", "Ay": "ay", "Saha Müşteri Grup": "grup_adi", "Ürün Kodu": "en_guncel_kod", "Koli İçi Adet": "koli_ici_adet"}, inplace=True)
btt_koli_ici_adet.rename(columns={"Yıl": "yil", "Ay": "ay", "Saha Müşteri Grup": "grup_adi", "Ürün Kodu": "en_guncel_kod", "Koli İçi Adet": "koli_ici_adet"}, inplace=True)
btt_koli_ici_adet["grup_adi"] = "BTT"
btt_koli_ici_adet = btt_koli_ici_adet[horizon_koli_ici_adet.columns.to_list()]

In [34]:
horizon_koli_ici_adet["tarih"] = horizon_koli_ici_adet["yil"].astype(str) + "-" + horizon_koli_ici_adet["ay"].astype(str) + "-01"
btt_koli_ici_adet["tarih"] = btt_koli_ici_adet["yil"].astype(str) + "-" + btt_koli_ici_adet["ay"].astype(str) + "-01"

horizon_koli_ici_adet["tarih"] = pd.to_datetime(horizon_koli_ici_adet["tarih"])
btt_koli_ici_adet["tarih"] = pd.to_datetime(btt_koli_ici_adet["tarih"])

horizon_koli_ici_adet.drop(columns=["yil", "ay"], axis=1, inplace=True)
btt_koli_ici_adet.drop(columns=["yil", "ay"], axis=1, inplace=True)

In [35]:
horizon_koli_ici_adet.grup_adi = horizon_koli_ici_adet.grup_adi.apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)
pasifik_koli_ici_adet.drop_duplicates("en_guncel_kod", ignore_index=True, inplace=True)

In [36]:
pasifik_koli_ici_adet2 = []
for grup_adii in ["A101", "BİM", "Diğer_Pasifik", "ŞOK", "MİGROS"]:
    tmp = pasifik_koli_ici_adet.copy()
    tmp["grup_adi"] = grup_adii
    tmp["tarih"] = datetime(params_["time_info_for_debugging"]["yil"], params_["time_info_for_debugging"]["ay"], 1)
    pasifik_koli_ici_adet2.append(tmp)

In [37]:
pasifik_koli_ici_adet2 = pd.concat(pasifik_koli_ici_adet2, ignore_index=True)

In [38]:
pasifik_koli_ici_adet2 = pasifik_koli_ici_adet2[horizon_koli_ici_adet.columns.to_list()]

In [39]:
koli_ici_adet_all = pd.concat([pasifik_koli_ici_adet2, horizon_koli_ici_adet, btt_koli_ici_adet], ignore_index=True, axis=0)

# Sadece Gerekli Sütunlar Tutuluyor

In [40]:
pasifik_df_all.drop(columns=["Organizasyon Kodu", "Grup Kodu.", "Pladis-Non Pladis"], axis=1, inplace=True)
horizon_saha_df_all.drop(columns=["Çeyrek"], axis=1, inplace=True)
btt_df_all.drop(columns=["Çeyrek", "Şirket Kodu"], axis=1, inplace=True)

btt_df_all["Grup Adı"] = "BTT"
pasifik_df_all.rename(columns={"Ana Kategori": "Ana Kategori Adı", "Kategori": "Kategori Adı", "Ürün Adı (Orjinal)": "Ürün Adı", "Sipariş Miktarı(Dönüş. Koli)": "Koli", 
                               "Sipariş Brüt Tutar": "KG", "Sipariş Brüt KG": "TL"}, inplace=True)

horizon_saha_df_all.rename(columns={"Horizon müşteri grup": "Grup Adı", "Ürün Adı (Orjinal)": "Ürün Adı"}, inplace=True)

In [41]:
horizon_saha_df_all.dropna(inplace=True, how="any")
horizon_saha_df_all.reset_index(inplace=True)

## Pasifik Kısmı

In [42]:
col_order = ['Yıl', 'Ay', 'Grup Adı', 'Ana Kategori Adı', 'Kategori Adı', 'Marka Adı', 'Ürün Kodu', 'Ürün Adı', 'Koli', 'KG', 'TL']
ltrs = list(string.ascii_letters) # Alfabede bulunan tüm harfleri tutan liste. Bunu, koli içi adet dataframe'deki harf içeren ürün kodlarını elemek için tutuyoruz.

horizon_saha_df_all["Kategori Adı"] = np.nan

pasifik_df_all = pasifik_df_all[col_order]
horizon_saha_df_all = horizon_saha_df_all[col_order]
btt_df_all = btt_df_all[col_order]

In [43]:
def read_aktivite_detay(paths_detay, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_detay+loop_list, skiprows=1, sheet_name="Ürün Detay", usecols="B:M")
    print("Okuma bitti ---->", loop_list)
    return new_df

In [44]:
def read_aktivite_cat(paths_cat, loop_list):
    import pandas as pd
    print("Okumaya başladı.", loop_list)
    new_df = pd.read_excel(paths_cat+loop_list, skiprows=1, sheet_name="Kategori", usecols="B:I")
    print("Okuma bitti ---->", loop_list)
    return new_df

In [45]:
start_time = datetime.now()

In [46]:
saha_aktivite_detay = []
#saha_aktivite_cat = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    
    paths_detay = params_["path"]["horizon_aktivite_path"]
#    paths_cat = params_["path"]["horizon_aktivite_path"]

    loop_detay = saha_aktivite_lst
#    loop_cat = saha_aktivite_lst
    
    func_detay = partial(read_aktivite_detay, paths_detay)
#    func_cat = partial(read_aktivite_cat, paths_cat)
    
    with mp.Pool(available_cpu) as p:
        saha_aktivite_detay.append(p.map(func_detay, loop_detay))
#    with mp.Pool(available_cpu) as p:
#        saha_aktivite_cat.append(p.map(func_cat, loop_cat))

In [47]:
end_time = datetime.now()
print('Dataların yüklenmesi: {}'.format(end_time - start_time))

Dataların yüklenmesi: 0:00:16.318386


In [48]:
saha_aktivite_detay = pd.concat(saha_aktivite_detay[0], ignore_index=True)
#saha_aktivite_cat = pd.concat(saha_aktivite_cat[0], ignore_index=True)

In [49]:
saha_aktivite_detay.drop_duplicates(subset=["Yıl", "Ay", "Saha Müşteri Grup", "Ürün Kodu"], keep="first", ignore_index=True, inplace=True)

In [50]:
# Read Pasifik Aktiviteleri
pasifik_aktivite_df = pd.read_excel(params_["path"]["pasifik_aktivite_path"]+pasifik_aktivite_lst[0])
pasifik_aktivite_df.drop_duplicates(subset=["Yıl", "Ay", "Müşteri Grup", "Ürün Kodu"], keep="first", ignore_index=True, inplace=True)

In [51]:
# Read Fiyat Listesi
fiyat_lst_pasifik = pd.read_excel(params_["path"]["fiyat_listesi_path"]+params_["files"]["pasifik_fiyat_file"])
fiyat_lst_horizon = pd.read_excel(params_["path"]["fiyat_listesi_path"]+params_["files"]["horizon_fiyat_file"])

In [52]:
# Read Portföy
pasifik_portfoy_df = pd.read_excel(params_["path"]["portfoy_path"]+portfoy_lst[0], sheet_name="Pasifik Portföy", skiprows=3, usecols="D:H")
btt_portfoy_df = pd.read_excel(params_["path"]["portfoy_path"]+portfoy_lst[0], sheet_name="BTT Portföy", skiprows=2, usecols="D:H")
horizon_portfoy_df = pd.read_excel(params_["path"]["portfoy_path"]+portfoy_lst[0], sheet_name="Horizon Portföy", skiprows=2, usecols="E:I")

In [53]:
# Read Eşlenik Kodları
eslenik_kod_df = pd.read_excel(params_["path"]["eslenik_kod_path"]+eslenik_kod_lst[0])

In [54]:
# Read Calender
calender_df = pd.read_excel(params_["path"]["calender_path"]+params_["files"]["calender_file"])

In [55]:
eslenik_kod_df["En Güncel Kod"] = eslenik_kod_df["En Güncel Kod"].apply(lambda x: int(x) if x not in ['delist ', "delist", "Delist"] else x.replace(" ", ""))

In [56]:
a101_kapsam = pd.read_excel(params_["path"]["kapsam_path"]+kapsam_listeli[0], sheet_name="A101 Portföy")
sok_kapsam = pd.read_excel(params_["path"]["kapsam_path"]+kapsam_listeli[0], sheet_name="Şok Portföy")
bim_kapsam = pd.read_excel(params_["path"]["kapsam_path"]+kapsam_listeli[0], sheet_name="Bim Portföy")

a101_kapsam["grup_adi"] = "A101"
sok_kapsam["grup_adi"] = "ŞOK"
bim_kapsam["grup_adi"] = "BİM"

kapsam_all = pd.concat([a101_kapsam, sok_kapsam, bim_kapsam], axis=0, ignore_index=True)

In [57]:
# Read Koli Birim Ağırlıkları
koli_birim_agirlik = pd.read_excel(params_["files"]["koli_agirlik_birim_file"])
koli_birim_agirlik_pas = pd.read_excel(params_["files"]["koli_agirlik_birim_file"])

In [58]:
koli_birim_agirlik.rename(columns={"Malzeme": "en_guncel_kod", 
                                   "Malzeme Açıklaması": "urun_adi", 
                                   "Ana Kategori": "ana_kategori_adi",
                                   "Kategori": "kategori_adi",
                                   "Alt Kategori": "alt_kategori_adi",
                                   "Detay Kategori": "detay_kategori_adi",
                                   "Marka": "marka_adi", "Marka Açıklaması": "marka_aciklamasi",
                                   "Net Ağırlık": "kg"}, inplace=True)

koli_birim_agirlik_pas.rename(columns={"Malzeme": "en_guncel_kod", 
                                   "Malzeme Açıklaması": "urun_adi", 
                                   "Ana Kategori": "ana_kategori_adi",
                                   "Kategori": "kategori_adi",
                                   "Alt Kategori": "alt_kategori_adi",
                                   "Detay Kategori": "detay_kategori_adi",
                                   "Marka": "marka_adi", "Marka Açıklaması": "marka_aciklamasi",
                                   "Net Ağırlık": "kg"}, inplace=True)

In [59]:
koli_birim_agirlik_pas["kanal"] = "pasifik"
koli_birim_agirlik_hor = koli_birim_agirlik.copy()
koli_birim_agirlik_btt = koli_birim_agirlik.copy()
koli_birim_agirlik_hor["kanal"] = "horizon"
koli_birim_agirlik_btt["kanal"] = "btt"

In [60]:
koli_birim_agirlik = pd.concat([koli_birim_agirlik_pas, koli_birim_agirlik_hor, koli_birim_agirlik_btt], axis=0, ignore_index=True)

In [61]:
end_time = datetime.now()
print('Diğer dataların Jupytere yüklenme süresi: {}'.format(end_time - start_time))

Diğer dataların Jupytere yüklenme süresi: 0:00:21.596053


---
# Pasifik 2016 aktivite verileri olmadığı için 2016 Sales dataları çıkartıldı.

In [62]:
pasifik_df_all = pasifik_df_all[pasifik_df_all["Yıl"] != 2016].reset_index(drop=True)

In [63]:
horizon_saha_df_all["Yıl"] = horizon_saha_df_all["Yıl"].astype(int)
horizon_saha_df_all["Ay"] = horizon_saha_df_all["Ay"].astype(int)

---
# Sales Datası İçin Ürün Kod Eşleme
---

## Join

In [64]:
pasifik_df_all = pd.merge(pasifik_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
horizon_saha_df_all = pd.merge(horizon_saha_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
btt_df_all = pd.merge(btt_df_all, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")

## Ürün Eşleme Kodları dosyasında yer almayan kodlar için mevcut ürün kodları verildi.

In [65]:
empty_ = pasifik_df_all[pd.isnull(pasifik_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = pasifik_df_all[~pd.isnull(pasifik_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
pasifik_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
pasifik_df_all = pasifik_df_all.sort_values(pasifik_df_all.columns.to_list()).reset_index(drop=True)

empty_ = btt_df_all[pd.isnull(btt_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = btt_df_all[~pd.isnull(btt_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
btt_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
btt_df_all = btt_df_all.sort_values(btt_df_all.columns.to_list()).reset_index(drop=True)

empty_ = horizon_saha_df_all[pd.isnull(horizon_saha_df_all["En Güncel Kod"])].reset_index(drop=True)
full_ = horizon_saha_df_all[~pd.isnull(horizon_saha_df_all["En Güncel Kod"])].reset_index(drop=True)
empty_["En Güncel Kod"] = empty_["Ürün Kodu"]
horizon_saha_df_all = pd.concat([empty_, full_], axis=0, ignore_index=True)
horizon_saha_df_all = horizon_saha_df_all.sort_values(horizon_saha_df_all.columns.to_list()).reset_index(drop=True)

## Adet adında yeni bir kolon oluşturuldu. Koli Sayısı 100'den az olanlara 0 yazıyoruz

In [66]:
pasifik_df_all["Koli"] = np.where(pasifik_df_all["Koli"] < 100, 0, pasifik_df_all["Koli"])
btt_df_all["Koli"] = np.where(btt_df_all["Koli"] < 100, 0, btt_df_all["Koli"])
horizon_saha_df_all["Koli"] = np.where(horizon_saha_df_all["Koli"] < 100, 0, horizon_saha_df_all["Koli"])

In [67]:
pasifik_df_all = pasifik_df_all[pasifik_df_all["Koli"] != 0].reset_index(drop=True)
btt_df_all = btt_df_all[btt_df_all["Koli"] != 0].reset_index(drop=True)
horizon_saha_df_all = horizon_saha_df_all[horizon_saha_df_all["Koli"] != 0].reset_index(drop=True)

## Delist olan ürünler veriden çıkartıldı.

In [68]:
pasifik_df_all = pasifik_df_all[pasifik_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)
btt_df_all = btt_df_all[btt_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)
horizon_saha_df_all = horizon_saha_df_all[horizon_saha_df_all["En Güncel Kod"] != "delist"].reset_index(drop=True)

## Aynı yıl, ay, grup adı, ana kategori adı, kategori adı, marka adı ve SKU kodundaki ürünler için toplam alındı. Sadece Koli İçi Adet için maksimum olan alındı.

---

# Marka adı dahil değil groupby'a
---

In [69]:
dct_to_sum = {"Koli": "sum", "KG": "sum", "TL": "sum"}

pasifik_df_all2 = pasifik_df_all.groupby(["Yıl", "Ay", "Grup Adı", "Ana Kategori Adı", "Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()
btt_df_all2 = btt_df_all.groupby(["Yıl", "Ay", "Grup Adı", "Ana Kategori Adı", "Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()
horizon_saha_df_all2 = horizon_saha_df_all.groupby(["Yıl", "Ay", "Grup Adı", "Ana Kategori Adı", "En Güncel Kod"]).agg(dct_to_sum).reset_index()

In [70]:
pasifik_df_all2["Date"] = pasifik_df_all2["Yıl"].astype(str) + "-" +  pasifik_df_all2["Ay"].astype(str) + "-01"
btt_df_all2["Date"] = btt_df_all2["Yıl"].astype(int).astype(str) + "-" +  btt_df_all2["Ay"].astype(int).astype(str) + "-01"
horizon_saha_df_all2["Date"] = horizon_saha_df_all2["Yıl"].astype(int).astype(str) + "-" +  horizon_saha_df_all2["Ay"].astype(int).astype(str) + "-01"

pasifik_df_all2["Date"] = pd.to_datetime(pasifik_df_all2["Date"], format="%Y-%m-%d", errors="coerce")
btt_df_all2["Date"] = pd.to_datetime(btt_df_all2["Date"], format="%Y-%m-%d", errors="coerce")
horizon_saha_df_all2["Date"] = pd.to_datetime(horizon_saha_df_all2["Date"], format="%Y-%m-%d", errors="coerce")

In [71]:
pasifik_df_all2.shape, horizon_saha_df_all2.shape, btt_df_all2.shape

((43165, 10), (173332, 9), (14501, 10))

## Horizon ve Pasifikte bulunan "Diğer"'lerin yanlarına "_" ile Diğer_Pasifik, Diğer_Horizon yazıldı.

In [72]:
pasifik_df_all2["Grup Adı"] = pasifik_df_all2["Grup Adı"].apply(lambda x: "Diğer_Pasifik" if x == "Diğer" else x)
horizon_saha_df_all2["Grup Adı"] = horizon_saha_df_all2["Grup Adı"].apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)

In [73]:
df_all2 = pd.concat([pasifik_df_all2, horizon_saha_df_all2, btt_df_all2], ignore_index=True)

In [74]:
if params_["time_info_for_debugging"]["ay"] == 12:
    ay_threshold = 1 #params_["time_info_for_debugging"]["ay"] = 1
    yil_threshold = params_["time_info_for_debugging"]["yil"] + 1
else:
    ay_threshold = params_["time_info_for_debugging"]["ay"] + 1
    yil_threshold = params_["time_info_for_debugging"]["yil"]

In [75]:
pasifik_df_all2 = pasifik_df_all2[pasifik_df_all2["Date"] < datetime(yil_threshold, ay_threshold, 1)].reset_index(drop=True)
horizon_saha_df_all2 = horizon_saha_df_all2[horizon_saha_df_all2["Date"] < datetime(yil_threshold, ay_threshold, 1)].reset_index(drop=True)
btt_df_all2 = btt_df_all2[btt_df_all2["Date"] < datetime(yil_threshold, ay_threshold, 1)].reset_index(drop=True)

---
# Pasifik Filling Missing Dates
---

In [76]:
def fill_missing_dates(df__, df_unique_list):
    df_all_filled = []
    for idx in df_unique_list.index:
        tmp_df = df__[(df__["Grup Adı"] == df_unique_list["Grup Adı"][idx]) & (df__["En Güncel Kod"] == df_unique_list["En Güncel Kod"][idx])]
        time_interval = []
        dt = tmp_df.Date.min()
        while dt <= datetime(params_["time_info_for_debugging"]["yil"], params_["time_info_for_debugging"]["ay"], 1):
            time_interval.append(dt)
            dt += relativedelta(months=1)
        date_to_add = [i for i in time_interval if i not in tmp_df.Date.unique()]
        if len(date_to_add) > 0:
            add_df = pd.concat([pd.DataFrame(tmp_df.iloc[0]).T]*len(date_to_add))
            add_df["Koli"], add_df["KG"], add_df["TL"] = 0, 0, 0
            add_df["Date"] = date_to_add
            add_df["Yıl"] = add_df["Date"].dt.year
            add_df["Ay"] = add_df["Date"].dt.month
            df_all_filled.append(pd.concat([tmp_df, add_df], ignore_index=True))
        else: 
            df_all_filled.append(tmp_df)
    return pd.concat(df_all_filled, ignore_index=True)

In [77]:
start_time = datetime.now()

In [78]:
unique_list = pasifik_df_all2.drop_duplicates(subset=["Grup Adı", "En Güncel Kod"], ignore_index=True)[["Grup Adı", "En Güncel Kod"]]
pasifik_df_all2 = fill_missing_dates(pasifik_df_all2, unique_list)

In [79]:
end_time = datetime.now()
print('Pasifik missing value düzenlenmesi süresi: {}'.format(end_time - start_time))

Pasifik missing value düzenlenmesi süresi: 0:00:33.762857


---
# Horizon Filling Missing Values
---

In [80]:
start_time = datetime.now()

In [81]:
unique_list = horizon_saha_df_all2.drop_duplicates(subset=["Grup Adı", "En Güncel Kod"], ignore_index=True)[["Grup Adı", "En Güncel Kod"]]
horizon_saha_df_all2 = fill_missing_dates(horizon_saha_df_all2, unique_list)

In [82]:
end_time = datetime.now()
print('Horizon missing value düzenlenmesi süresi: {}'.format(end_time - start_time))

Horizon missing value düzenlenmesi süresi: 0:05:08.159233


---
# BTT Filling Missing Values
---

In [83]:
start_time = datetime.now()

In [84]:
unique_list = btt_df_all2.drop_duplicates(subset=["Grup Adı", "En Güncel Kod"], ignore_index=True)[["Grup Adı", "En Güncel Kod"]]
btt_df_all2 = fill_missing_dates(btt_df_all2, unique_list)

In [85]:
end_time = datetime.now()
print('BTT missing value düzenlenmesi süresi: {}'.format(end_time - start_time))

BTT missing value düzenlenmesi süresi: 0:00:05.222167


In [86]:
df_all22 = pd.concat([pasifik_df_all2, horizon_saha_df_all2, btt_df_all2], ignore_index=True)

---

# Aktivite Datası İçin Ürün Kod Eşleme

---

# Pasifik Aktivite

## Left join ile güncel kodlar getirildi. Delist olan ürünler listeden çıkartıldı. "Çeyrek" sütunu silindi. En güncel kod sütunnuda bulunamayan değerler Ürün Kodu sütunundan çekildi.

In [87]:
pasifik_aktivite_df2 = pd.merge(pasifik_aktivite_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
pasifik_aktivite_df2 = pasifik_aktivite_df2[pasifik_aktivite_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
pasifik_aktivite_df2.drop("Çeyrek", inplace=True, axis=1)
pasifik_aktivite_df2['En Güncel Kod'] = pasifik_aktivite_df2['En Güncel Kod'].fillna(pasifik_aktivite_df2['Ürün Kodu'])
pasifik_aktivite_df2.drop(columns="Ürün Kodu", axis=1, inplace=True)

---

# Pasifik Aktivite Ciro - Promosyon Tutarı ve İskonto Tekilleştirme (ORTALAMA ALARAK)

In [88]:
ort_pas = {"Raf Tavsiye Satış Fiyatı": "mean", "İndirimli Raf Satış Fiyatı": "mean", "İndirim %": "mean", "Aktivite Tipi": "first"}
pasifik_aktivite_df3 = pasifik_aktivite_df2.groupby(["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup"]).agg(ort_pas).reset_index()
pasifik_aktivite_df3 = pd.merge(pasifik_aktivite_df3, pasifik_aktivite_df2[["Yıl", "Ay", "Müşteri Grup", "En Güncel Kod", 
                                                                            "Ana Kategori Adı", "Kategori Adı", "Marka Adı"]],
                                how="left", 
                                on=["En Güncel Kod", "Yıl", "Ay", "Müşteri Grup"])

pasifik_aktivite_df3.drop_duplicates(subset=pasifik_aktivite_df3.columns.to_list(), inplace=True)
pasifik_aktivite_df3.reset_index(drop=True, inplace=True)
pasifik_aktivite_df3 = pasifik_aktivite_df3[pasifik_aktivite_df2.drop("Ürün Adı", axis=1).columns.to_list()]

---

# Horizon Aktivite

In [89]:
saha_aktivite_detay2 = pd.merge(saha_aktivite_detay, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], on="Ürün Kodu", how="left")
saha_aktivite_detay2 = saha_aktivite_detay2[saha_aktivite_detay2["En Güncel Kod"] != "delist"].reset_index(drop=True)
saha_aktivite_detay2.drop("Çeyrek", inplace=True, axis=1)
saha_aktivite_detay2['En Güncel Kod'] = saha_aktivite_detay2['En Güncel Kod'].fillna(saha_aktivite_detay2['Ürün Kodu'])
saha_aktivite_detay2.drop(columns="Ürün Kodu", axis=1, inplace=True)
saha_aktivite_detay2["İskonto %"].replace("#DIV/0", np.nan,inplace=True)

# Horizon Aktivite Ciro - Promosyon Tutarı ve İskonto Tekilleştirme (ORTALAMA ALARAK)

In [90]:
ort = {"Ciro (Kull. İade Düş.)": "mean", "Promosyon Tutarı": "mean", "İskonto %": "mean"}
saha_aktivite_detay3 = saha_aktivite_detay2.groupby(["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup"]).agg(ort).reset_index()

saha_aktivite_detay3 = pd.merge(saha_aktivite_detay3, saha_aktivite_detay2[["Yıl", "Ay", "Saha Müşteri Grup", "En Güncel Kod", 
                                                 "Ana Kategori Adı", "Kategori Adı", "Marka Adı"]],
                           how="left", 
                           on=["En Güncel Kod", "Yıl", "Ay", "Saha Müşteri Grup"])

saha_aktivite_detay3.drop_duplicates(subset=saha_aktivite_detay3.columns.to_list(), inplace=True)

In [91]:
saha_aktivite_detay3 = saha_aktivite_detay3[saha_aktivite_detay2.drop("Ürün Adı (Mobis)", axis=1).columns.to_list()].reset_index(drop=True)

In [92]:
saha_aktivite_detay3.rename(columns={"Saha Müşteri Grup": "Grup Adı"}, inplace=True)
saha_aktivite_detay3["Grup Adı"] = saha_aktivite_detay3["Grup Adı"].apply(lambda x: "Diğer_Horizon" if x == "Diğer" else x)

---

# Fiyat Listesi

## Horizon Fiyatları
---

In [93]:
# Read Fiyat Listesi
fiyat_lst_horizon = pd.read_excel(params_["path"]["fiyat_listesi_path"]+params_["files"]["horizon_fiyat_file"])

In [94]:
start_time = datetime.now()

In [95]:
fiyat_lst_horizon.drop_duplicates(subset=fiyat_lst_horizon.columns.to_list(), keep="first", ignore_index=True, inplace=True)

In [96]:
try:
    fiyat_lst_horizon["Malzeme"] = fiyat_lst_horizon["Malzeme"].str.replace("-", "")
    fiyat_lst_horizon["Malzeme"] = fiyat_lst_horizon["Malzeme"].astype(int)*1
except:
    fiyat_lst_horizon["Malzeme"] = fiyat_lst_horizon["Malzeme"].astype(int)*1

In [97]:
fiyat_lst_horizon_df = fiyat_lst_horizon.copy()

In [98]:
fiyat_lst_horizon_df["Baslangic_Yıl"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.year)
fiyat_lst_horizon_df["Baslangic_Ay"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.month)
fiyat_lst_horizon_df["Baslangic_Gun"] = fiyat_lst_horizon_df["Bşl.tarihi"].apply(lambda x: x.day)
fiyat_lst_horizon_df["Gecerlilik_Yıl"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.year)
fiyat_lst_horizon_df["Gecerlilik_Ay"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.month)
fiyat_lst_horizon_df["Gecerlilik_Gun"] = fiyat_lst_horizon_df["Gçrl.sonu"].apply(lambda x: x.day)

In [99]:
fiyat_lst_horizon_df["Baslangic_Yıl"] = fiyat_lst_horizon_df["Baslangic_Yıl"].apply(lambda x: (horizon_saha_df_all2["Date"].max().year)+1 if x > horizon_saha_df_all2["Date"].max().year else x)
fiyat_lst_horizon_df["Gecerlilik_Yıl"] = fiyat_lst_horizon_df["Gecerlilik_Yıl"].apply(lambda x: (horizon_saha_df_all2["Date"].max().year)+1 if x > horizon_saha_df_all2["Date"].max().year else x)

In [100]:
time_index = horizon_saha_df_all2["Date"].unique()

In [101]:
h_fiyat_unique = []

for malzeme in fiyat_lst_horizon_df["Malzeme"].unique():
#for malzeme in [35903]:
    temp_time_df = pd.DataFrame({"Fiyat": [np.nan]}, index=time_index)
    temp_time_df = temp_time_df.reset_index().rename(columns={"index":"date"})    
    temp_time_df["En Güncel Kod"] = malzeme
    temp_time_df["fiyat_gecisi"] = 0
    malzeme_df = fiyat_lst_horizon_df[fiyat_lst_horizon_df["Malzeme"] == malzeme].reset_index(drop=True)
    malzeme_df.drop(columns=["KşTü", "Koşul türü", "Tanım", "Ana Kategori", "Kategori", "ÖB"], axis=1, inplace=True)
    malzeme_df.drop_duplicates(subset=malzeme_df.columns.to_list(), inplace=True, ignore_index=True)
    malzeme_df.sort_values(by=["Baslangic_Yıl", "Baslangic_Ay", "Baslangic_Gun"], ignore_index=True, inplace=True)
    check_idx1 = []
    if len(malzeme_df) > 1:
        for row1 in malzeme_df.index:
            for row2 in malzeme_df[row1+1:].index:
                if (malzeme_df.loc[row1]["Gecerlilik_Yıl"] == malzeme_df.loc[row2]["Baslangic_Yıl"]) and (malzeme_df.loc[row1]["Gecerlilik_Ay"] == malzeme_df.loc[row2]["Baslangic_Ay"]):
                    num_days = calendar.monthrange(int(malzeme_df.loc[row2]["Baslangic_Yıl"]), int(malzeme_df.loc[row2]["Baslangic_Ay"]))[1]
                    fyt=((int(malzeme_df.loc[row1]["Gecerlilik_Gun"])*malzeme_df.loc[row1]["     Tutar"]) + (num_days - int(malzeme_df.loc[row2]["Baslangic_Gun"]) + 1)*malzeme_df.loc[row2]["     Tutar"])/num_days

                    end_idx1 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx1 = temp_time_df[temp_time_df["date"] == end_idx1].index
                    temp_time_df.loc[final_idx1, "Fiyat"] = fyt
                    temp_time_df.loc[final_idx1,"fiyat_gecisi"] = 1

                elif (malzeme_df.loc[row1, "Gecerlilik_Gun"] == calendar.monthrange(int(malzeme_df.loc[row1]["Gecerlilik_Yıl"]), int(malzeme_df.loc[row1]["Gecerlilik_Ay"]))[1] \
                     and malzeme_df.loc[row2, "Baslangic_Gun"] == 1):
                    fyt5=malzeme_df.loc[row1]["     Tutar"]
                    fyt6=malzeme_df.loc[row2]["     Tutar"]
                    end_idx5 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    end_idx6 =  datetime(int(malzeme_df.loc[row2, "Baslangic_Yıl"]), int(malzeme_df.loc[row2, "Baslangic_Ay"]), 1)
                    final_idx5 = temp_time_df[temp_time_df["date"] == end_idx5].index
                    final_idx6 = temp_time_df[temp_time_df["date"] == end_idx6].index
                    temp_time_df.loc[final_idx5, "Fiyat"] = fyt5
                    temp_time_df.loc[final_idx6, "Fiyat"] = fyt6

                else:
                    fyt2=malzeme_df.loc[row1]["     Tutar"]
                    start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx2 = temp_time_df[(temp_time_df["date"] > start_idx2) & (temp_time_df["date"] < end_idx2)].index
                    temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
            if (row1 == len(malzeme_df)-1) or (row1 == len(malzeme_df)-2):
                fyt3=malzeme_df.loc[row1]["     Tutar"]
                start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                final_idx3 = temp_time_df[(temp_time_df["date"] > start_idx3) & (temp_time_df["date"] < end_idx3)].index
                temp_time_df.loc[final_idx3, "Fiyat"] = fyt3

    else:
        for row1 in malzeme_df.index:
            fyt4=malzeme_df.loc[row1]["     Tutar"]
            start_idx4 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
            end_idx4 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
            final_idx4 = temp_time_df[(temp_time_df["date"] >= start_idx4) & (temp_time_df["date"] <= end_idx4)].index
            temp_time_df.loc[final_idx4, "Fiyat"] = fyt4

    if (malzeme_df.loc[0, "Baslangic_Yıl"] >= min(horizon_saha_df_all2["Yıl"].unique())) and (len(malzeme_df) > 1):
        temp_time_df.loc[temp_time_df[~pd.isnull(temp_time_df["Fiyat"])].index[0]-1, "Fiyat"] = malzeme_df.loc[0, "     Tutar"]
    temp_time_df = temp_time_df.dropna().reset_index(drop=True)

    h_fiyat_unique.append(temp_time_df)

In [102]:
h_fiyat_unique = pd.concat(h_fiyat_unique)
h_fiyat_unique.reset_index(drop=True, inplace=True)

h_fiyat_unique.rename(columns={"En Güncel Kod": "Ürün Kodu", "date": "Date"}, inplace=True)
h_fiyat_unique = h_fiyat_unique.merge(eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left")
h_fiyat_unique["En Güncel Kod"].fillna(h_fiyat_unique["Ürün Kodu"], inplace=True)
h_fiyat_unique = h_fiyat_unique[h_fiyat_unique["En Güncel Kod"] != "delist"].reset_index(drop=True)
h_fiyat_unique = h_fiyat_unique.sort_values(by=["En Güncel Kod", "Date"]).reset_index(drop=True)
h_fiyat_unique = h_fiyat_unique.drop(columns="Ürün Kodu", axis=1)
# Aynı aya denk gelen ürünlerin fiyatlarının ortalaması alınıp, herhangi birinde fiyat geçişi varsa 1 alınır.
h_fiyat_unique = h_fiyat_unique.groupby(["Date", "En Güncel Kod"]).agg({"Fiyat": "mean", "fiyat_gecisi": "max"}).reset_index()

In [103]:
end_time = datetime.now()
print('Horizon fiyatların düzenlenmesi süresi: {}'.format(end_time - start_time))

Horizon fiyatların düzenlenmesi süresi: 0:02:30.665185


## Pasifik Fiyatları
---

In [104]:
fiyat_lst_pasifik = pd.read_excel(params_["path"]["fiyat_listesi_path"]+params_["files"]["pasifik_fiyat_file"])

In [105]:
start_time = datetime.now()

In [106]:
fiyat_lst_pasifik.drop_duplicates(subset=fiyat_lst_pasifik.columns.to_list(), keep="first", ignore_index=True, inplace=True)

In [107]:
fiyat_lst_pasifik["Malzeme"] = fiyat_lst_pasifik["Malzeme"].str.replace("-", "")
fiyat_lst_pasifik["Malzeme"] = fiyat_lst_pasifik["Malzeme"].astype(int)*1

In [108]:
fiyat_lst_pasifik_df = fiyat_lst_pasifik.copy()

In [109]:
fiyat_lst_pasifik_df["Baslangic_Yıl"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.year)
fiyat_lst_pasifik_df["Baslangic_Ay"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.month)
fiyat_lst_pasifik_df["Baslangic_Gun"] = fiyat_lst_pasifik_df["Bşl.tarihi"].apply(lambda x: x.day)
fiyat_lst_pasifik_df["Gecerlilik_Yıl"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.year)
fiyat_lst_pasifik_df["Gecerlilik_Ay"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.month)
fiyat_lst_pasifik_df["Gecerlilik_Gun"] = fiyat_lst_pasifik_df["Gçrl.sonu"].apply(lambda x: x.day)

In [110]:
fiyat_lst_pasifik_df["Baslangic_Yıl"] = fiyat_lst_pasifik_df["Baslangic_Yıl"].apply(lambda x: (pasifik_df_all2["Date"].max().year)+1 if x > pasifik_df_all2["Date"].max().year else x)
fiyat_lst_pasifik_df["Gecerlilik_Yıl"] = fiyat_lst_pasifik_df["Gecerlilik_Yıl"].apply(lambda x: (pasifik_df_all2["Date"].max().year)+1 if x > pasifik_df_all2["Date"].max().year else x)

In [111]:
time_index = pasifik_df_all2["Date"].unique()

In [112]:
p_fiyat_unique = []

for malzeme in fiyat_lst_pasifik_df["Malzeme"].unique():
    temp_time_df = pd.DataFrame({"Fiyat": [np.nan]}, index=time_index)
    temp_time_df = temp_time_df.reset_index().rename(columns={"index":"date"})    
    temp_time_df["En Güncel Kod"] = malzeme
    temp_time_df["fiyat_gecisi"] = 0
    malzeme_df = fiyat_lst_pasifik_df[fiyat_lst_pasifik_df["Malzeme"] == malzeme].reset_index(drop=True)
    malzeme_df.drop(columns=["KşTü", "KşTü.1", "Malzeme Tanım", "Ana Kategori", "Kategori"], axis=1, inplace=True)
    malzeme_df.drop_duplicates(subset=malzeme_df.columns.to_list(), inplace=True, ignore_index=True)
    malzeme_df.sort_values(by=["Baslangic_Yıl", "Baslangic_Ay", "Baslangic_Gun"], ignore_index=True, inplace=True)
    check_idx1 = []
    if len(malzeme_df) > 1:
        for row1 in malzeme_df.index:
            for row2 in malzeme_df[row1+1:].index:
                if (malzeme_df.loc[row1]["Gecerlilik_Yıl"] == malzeme_df.loc[row2]["Baslangic_Yıl"]) and (malzeme_df.loc[row1]["Gecerlilik_Ay"] == malzeme_df.loc[row2]["Baslangic_Ay"]):
                    num_days = calendar.monthrange(int(malzeme_df.loc[row2]["Baslangic_Yıl"]), int(malzeme_df.loc[row2]["Baslangic_Ay"]))[1]
                    fyt=((int(malzeme_df.loc[row1]["Gecerlilik_Gun"])*malzeme_df.loc[row1]["Koli TL"]) + (num_days - int(malzeme_df.loc[row2]["Baslangic_Gun"])+1)*malzeme_df.loc[row2]["Koli TL"])/num_days

                    end_idx1 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx1 = temp_time_df[temp_time_df["date"] == end_idx1].index
                    temp_time_df.loc[final_idx1, "Fiyat"] = fyt
                    temp_time_df.loc[final_idx1,"fiyat_gecisi"] = 1

                elif (malzeme_df.loc[row1, "Gecerlilik_Gun"] == calendar.monthrange(int(malzeme_df.loc[row1]["Gecerlilik_Yıl"]), int(malzeme_df.loc[row1]["Gecerlilik_Ay"]))[1] \
                     and malzeme_df.loc[row2, "Baslangic_Gun"] == 1):
                    fyt5=malzeme_df.loc[row1]["Koli TL"]
                    fyt6=malzeme_df.loc[row2]["Koli TL"]
                    end_idx5 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    end_idx6 =  datetime(int(malzeme_df.loc[row2, "Baslangic_Yıl"]), int(malzeme_df.loc[row2, "Baslangic_Ay"]), 1)
                    final_idx5 = temp_time_df[temp_time_df["date"] == end_idx5].index
                    final_idx6 = temp_time_df[temp_time_df["date"] == end_idx6].index
                    temp_time_df.loc[final_idx5, "Fiyat"] = fyt5
                    temp_time_df.loc[final_idx6, "Fiyat"] = fyt6


                else:
                    if malzeme_df.loc[row1, "Baslangic_Gun"] != 1:
                        fyt2=malzeme_df.loc[row1]["Koli TL"]
                        start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                        end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                        final_idx2 = temp_time_df[(temp_time_df["date"] > start_idx2) & (temp_time_df["date"] < end_idx2)].index
                        temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
                    else:
                        fyt2=malzeme_df.loc[row1]["Koli TL"]
                        start_idx2 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                        end_idx2 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                        final_idx2 = temp_time_df[(temp_time_df["date"] >= start_idx2) & (temp_time_df["date"] < end_idx2)].index
                        temp_time_df.loc[final_idx2, "Fiyat"] = fyt2
                        
            if (row1 == len(malzeme_df)-1) or (row1 == len(malzeme_df)-2):
                if malzeme_df.loc[row1, "Baslangic_Gun"] != 1:
                    fyt3=malzeme_df.loc[row1]["Koli TL"]
                    start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx3 = temp_time_df[(temp_time_df["date"] > start_idx3) & (temp_time_df["date"] < end_idx3)].index
                    temp_time_df.loc[final_idx3, "Fiyat"] = fyt3
                else:
                    fyt3=malzeme_df.loc[row1]["Koli TL"]
                    start_idx3 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
                    end_idx3 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
                    final_idx3 = temp_time_df[(temp_time_df["date"] >= start_idx3) & (temp_time_df["date"] < end_idx3)].index
                    temp_time_df.loc[final_idx3, "Fiyat"] = fyt3
    else:
        for row1 in malzeme_df.index:
            fyt4=malzeme_df.loc[row1]["Koli TL"]
            start_idx4 = datetime(int(malzeme_df.loc[row1, "Baslangic_Yıl"]), int(malzeme_df.loc[row1, "Baslangic_Ay"]), 1)
            end_idx4 =  datetime(int(malzeme_df.loc[row1, "Gecerlilik_Yıl"]), int(malzeme_df.loc[row1, "Gecerlilik_Ay"]), 1)
            final_idx4 = temp_time_df[(temp_time_df["date"] >= start_idx4) & (temp_time_df["date"] <= end_idx4)].index
            temp_time_df.loc[final_idx4, "Fiyat"] = fyt4
    
    if (malzeme_df.loc[0, "Baslangic_Yıl"] >= min(pasifik_df_all2["Yıl"].unique())) and (len(malzeme_df) > 1):
        temp_time_df.loc[temp_time_df[~pd.isnull(temp_time_df["Fiyat"])].index[0]-1, "Fiyat"] = malzeme_df.loc[0, "Koli TL"]
    temp_time_df = temp_time_df.dropna().reset_index(drop=True)
        
    p_fiyat_unique.append(temp_time_df)

In [113]:
p_fiyat_unique = pd.concat(p_fiyat_unique)
p_fiyat_unique.reset_index(drop=True, inplace=True)

p_fiyat_unique.rename(columns={"En Güncel Kod": "Ürün Kodu", "date": "Date"}, inplace=True)
p_fiyat_unique = p_fiyat_unique.merge(eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left")
p_fiyat_unique["En Güncel Kod"].fillna(p_fiyat_unique["Ürün Kodu"], inplace=True)
p_fiyat_unique = p_fiyat_unique[p_fiyat_unique["En Güncel Kod"] != "delist"].reset_index(drop=True)
p_fiyat_unique = p_fiyat_unique.sort_values(by=["En Güncel Kod", "Date"]).reset_index(drop=True)
p_fiyat_unique = p_fiyat_unique.drop(columns="Ürün Kodu", axis=1)
# Aynı aya denk gelen ürünlerin fiyatlarının ortalaması alınıp, herhangi birinde fiyat geçişi varsa 1 alınır.
p_fiyat_unique = p_fiyat_unique.groupby(["Date", "En Güncel Kod"]).agg({"Fiyat": "mean", "fiyat_gecisi": "max"}).reset_index()

In [114]:
end_time = datetime.now()
print('Pasifik fiyatların düzenlenmesi süresi: {}'.format(end_time - start_time))

Pasifik fiyatların düzenlenmesi süresi: 0:02:19.669212


---
# Portföy
---

## Pasifik Portföy

In [115]:
pasifik_portfoy_df2 = pd.merge(pasifik_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
pasifik_portfoy_df2["En Güncel Kod"] = pasifik_portfoy_df2["En Güncel Kod"].fillna(pasifik_portfoy_df2["Kod"])
pasifik_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
pasifik_portfoy_df2 = pasifik_portfoy_df2[pasifik_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
pasifik_portfoy_df2["Portfoy"] = 1

## Horizon Portföy

In [116]:
horizon_portfoy_df2 = pd.merge(horizon_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
horizon_portfoy_df2["En Güncel Kod"] = horizon_portfoy_df2["En Güncel Kod"].fillna(horizon_portfoy_df2["Kod"])
horizon_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
horizon_portfoy_df2 = horizon_portfoy_df2[horizon_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
horizon_portfoy_df2["Portfoy"] = 1
horizon_portfoy_df2 = horizon_portfoy_df2[~((horizon_portfoy_df2["Kod"] == 135901))]

## BTT Portföy

In [117]:
btt_portfoy_df2 = pd.merge(btt_portfoy_df, eslenik_kod_df[["Ürün Kodu", "En Güncel Kod"]], how="left", left_on="Kod", right_on="Ürün Kodu")
btt_portfoy_df2["En Güncel Kod"] = btt_portfoy_df2["En Güncel Kod"].fillna(btt_portfoy_df2["Kod"])
btt_portfoy_df2.drop("Ürün Kodu", axis=1, inplace=True)
btt_portfoy_df2 = btt_portfoy_df2[btt_portfoy_df2["En Güncel Kod"] != "delist"].reset_index(drop=True)
btt_portfoy_df2["Portfoy"] = 1
btt_portfoy_df2 = btt_portfoy_df2[~((btt_portfoy_df2["Kod"] == 135901))]

---

# Portföy Kapsamındaki Sales Dataları

---

In [118]:
pasifik_df_all3 = pd.merge(pasifik_df_all2,pasifik_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")
btt_df_all3 = pd.merge(btt_df_all2,btt_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")
horizon_saha_df_all3 = pd.merge(horizon_saha_df_all2, horizon_portfoy_df2[["En Güncel Kod", "Portfoy"]], on="En Güncel Kod", how="left")
pasifik_df_all3["Portfoy"].fillna(0, inplace=True)
btt_df_all3["Portfoy"].fillna(0, inplace=True)
horizon_saha_df_all3["Portfoy"].fillna(0, inplace=True)

In [119]:
pasifik_df_all3.shape, btt_df_all3.shape, horizon_saha_df_all3.shape

((65271, 11), (17549, 11), (331517, 10))

In [120]:
pasifik_df_all3.drop_duplicates(ignore_index=True, inplace=True)
btt_df_all3.drop_duplicates(ignore_index=True, inplace=True)
horizon_saha_df_all3.drop_duplicates(ignore_index=True, inplace=True)

---
# Datalara Calender Eklenmesi
---

In [121]:
calender_df.pop("DATE")

0     Jan 2016
1     Feb 2016
2     Mar 2016
3     Apr 2016
4     May 2016
5     Jun 2016
6     Jul 2016
7     Aug 2016
8     Sep 2016
9     Oct 2016
10    Nov 2016
11    Dec 2016
12    Jan 2017
13    Feb 2017
14    Mar 2017
15    Apr 2017
16    May 2017
17    Jun 2017
18    Jul 2017
19    Aug 2017
20    Sep 2017
21    Oct 2017
22    Nov 2017
23    Dec 2017
24    Jan 2018
25    Feb 2018
26    Mar 2018
27    Apr 2018
28    May 2018
29    Jun 2018
30    Jul 2018
31    Aug 2018
32    Sep 2018
33    Oct 2018
34    Nov 2018
35    Dec 2018
36    Jan 2019
37    Feb 2019
38    Mar 2019
39    Apr 2019
40    May 2019
41    Jun 2019
42    Jul 2019
43    Aug 2019
44    Sep 2019
45    Oct 2019
46    Nov 2019
47    Dec 2019
48    Jan 2020
49    Feb 2020
50    Mar 2020
51    Apr 2020
52    May 2020
53    Jun 2020
54    Jul 2020
55    Aug 2020
56    Sep 2020
57    Oct 2020
58    Nov 2020
59    Dec 2020
60    Jan 2021
61    Feb 2021
62    Mar 2021
63    Apr 2021
64    May 2021
65    Jun 2021
66    Jul 

In [122]:
pasifik_df_all3 = pd.merge(pasifik_df_all3, calender_df, on=["Yıl", "Ay"], how="left")
btt_df_all3 = pd.merge(btt_df_all3, calender_df, on=["Yıl", "Ay"], how="left")
horizon_saha_df_all3 = pd.merge(horizon_saha_df_all3, calender_df, on=["Yıl", "Ay"], how="left")

---
# Dataların Fiyat Ve Aktiviteler İle Birleştirilmesi
---

# Not:
---
### 1) BTT aktivite verisi için Horizon kısmındaki "Geleneksel Kanal" kullanılması istendi.
### 2) BTT fiyat geçişleri için Horizon fiyat geçişleri baz alındı.

In [123]:
pasifik_df_all3.shape, horizon_saha_df_all3.shape, btt_df_all3.shape

((62095, 28), (331147, 27), (17549, 28))

In [124]:
pasifik_df_all3 = pasifik_df_all3.merge(p_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])
pasifik_aktivite_df3.rename(columns={"Müşteri Grup": "Grup Adı", "Grup adı": "Grup Adı"}, inplace=True)
pasifik_df_all3 = pd.merge(pasifik_df_all3, pasifik_aktivite_df3[["Yıl", "Ay", "Grup Adı", "En Güncel Kod", 
                                                                  "Raf Tavsiye Satış Fiyatı", "İndirimli Raf Satış Fiyatı", "İndirim %",
                                                                  "Aktivite Tipi"]], 
                           left_on=["Yıl", "Ay", "Grup Adı", "En Güncel Kod"], 
                           right_on=["Yıl", "Ay", "Grup Adı", "En Güncel Kod"], how="left")

In [125]:
saha_aktivite_detay3.rename(columns={"Grup adı": "Grup Adı"}, inplace=True)

In [126]:
horizon_saha_df_all3 = horizon_saha_df_all3.merge(h_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])
horizon_saha_df_all3 = horizon_saha_df_all3.merge(saha_aktivite_detay3[['Ciro (Kull. İade Düş.)', 'Promosyon Tutarı', 
                                                                        'İskonto %', 'En Güncel Kod', "Yıl", "Ay", "Grup Adı"]],
                                                  on=["En Güncel Kod", "Yıl", "Ay", "Grup Adı"], how="left")

In [127]:
horizon_saha_df_all3.drop_duplicates(ignore_index=True, inplace=True)

In [128]:
btt_df_all3 = btt_df_all3.merge(h_fiyat_unique, how="left", on=["Date", "En Güncel Kod"])
btt_aktivite = saha_aktivite_detay3[saha_aktivite_detay3["Grup Adı"] == "GELENEKSEL KANAL"].reset_index(drop=True)

btt_df_all3 = btt_df_all3.merge(btt_aktivite[['Ciro (Kull. İade Düş.)', 'Promosyon Tutarı', 
                                              'İskonto %', 'En Güncel Kod', "Yıl", "Ay"]],
                                on=["En Güncel Kod", "Yıl", "Ay"], how="left")

In [129]:
btt_df_all3.drop_duplicates(ignore_index=True, inplace=True)

# Sütun İsimlerini İngilizce Karaktere Çevirme

In [130]:
def change_col_name(dff_):
    """
    Parameters:
    ----------
    
    dff_: dataframe
    Sütun ismini değiştirmek istediğiniz dataframe'i yazınız.
    
    Returns: Liste
    """
    
    
    chng_letters = list(zip(["ç", "ğ", "ı", "ö", "ş", "ü", " ", "%", ".", "(", ")", "-"], 
                            ["c", "g", "i", "o", "s", "u", "_", "", "", "", "", "_"]))
    new_cols = []
    for col in dff_.columns.str.lower():
        for letter in range(len(chng_letters)):
            col = col.replace(chng_letters[letter][0], chng_letters[letter][1])
            if letter == len(chng_letters) - 1:
                new_cols.append(col)
            else:
                pass
    return new_cols

In [131]:
pasifik_df_all3.columns = change_col_name(pasifik_df_all3)
horizon_saha_df_all3.columns = change_col_name(horizon_saha_df_all3)
btt_df_all3.columns = change_col_name(btt_df_all3)

---
# Pasifikte Aktivite Tipi Verisi Eksik Olan Verilere "Yok" yazıldı
---

In [132]:
pasifik_df_all3["aktivite_tipi"].fillna("Yok", inplace=True)

In [133]:
pasifik_df_sorted = pasifik_df_all3.copy()
horizon_saha_df_sorted = horizon_saha_df_all3.copy()
btt_df_sorted = btt_df_all3.copy()

In [134]:
df_pasifik = pasifik_df_sorted.copy()
df_btt = btt_df_sorted.copy()
df_horizon = horizon_saha_df_sorted.copy()

---
# Sütun isim uzunluğunun 32'yi geçmemesi için
---

In [135]:
df_pasifik.columns = [i[:32] if len(i) > 32 else i for i in df_pasifik.columns]
df_horizon.columns = [i[:32] if len(i) > 32 else i for i in df_horizon.columns]
df_btt.columns = [i[:32] if len(i) > 32 else i for i in df_btt.columns]

In [136]:
p_fiyat_unique.rename(columns={"Date": "date", "En Güncel Kod": "en_guncel_kod", "Fiyat": "fiyat"}, inplace=True)
h_fiyat_unique.rename(columns={"Date": "date", "En Güncel Kod": "en_guncel_kod", "Fiyat": "fiyat"}, inplace=True)

In [137]:
p_fiyat_unique["kanal"] = "pasifik"
h_fiyat_unique["kanal"] = "horizon"
fiyat_unique = pd.concat([p_fiyat_unique, h_fiyat_unique], axis=0, ignore_index=True)
#fiyat_unique.to_csv("../data/fiyat_list.csv", index=False)

In [138]:
df_pasifik3 = df_pasifik.copy()
df_btt3 = df_btt.copy()
df_horizon3 = df_horizon.copy()

In [139]:
df_pasifik4 = df_pasifik3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_pasifik4 = df_pasifik4.merge(p_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

df_horizon4 = df_horizon3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_horizon4 = df_horizon4.merge(h_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

df_btt4 = df_btt3.drop(columns=['fiyat', 'fiyat_gecisi'], axis=1)
df_btt4 = df_btt4.merge(h_fiyat_unique, how="left", on=["date", "en_guncel_kod"])

In [140]:
pasifik_aktivite_df3.columns = change_col_name(pasifik_aktivite_df3)
saha_aktivite_detay3.columns = change_col_name(saha_aktivite_detay3)
pasifik_aktivite_df3.rename(columns={"i̇ndirim_": "indirim_"}, inplace=True)
saha_aktivite_detay3.rename(columns={"i̇i̇skonto_": "iskonto_"}, inplace=True)

In [141]:
df_pasifik4 = df_pasifik4.drop(columns=['raf_tavsiye_satis_fiyati', 'i̇ndirimli_raf_satis_fiyati', 
                                        'i̇ndirim_', 'aktivite_tipi'], axis=1)

df_horizon4 = df_horizon4.drop(columns=['ciro_kull_i̇ade_dus', 'promosyon_tutari', 'i̇skonto_'], axis=1)
df_btt4 = df_btt4.drop(columns=['ciro_kull_i̇ade_dus', 'promosyon_tutari', 'i̇skonto_'], axis=1)

pasifik_aktivite_df3.rename(columns={"Müşteri Grup": "Grup adı"}, inplace=True)

In [142]:
df_pasifik5 = pd.merge(df_pasifik4, pasifik_aktivite_df3[['yil', 'ay', 'grup_adi', 'en_guncel_kod', 
                                                          'raf_tavsiye_satis_fiyati', 'i̇ndirimli_raf_satis_fiyati', 'indirim_', 
                                                          'aktivite_tipi']], 
                           left_on=['yil', 'ay', 'grup_adi', 'en_guncel_kod'], 
                           right_on=['yil', 'ay', 'grup_adi', 'en_guncel_kod'], how="left")

df_horizon5 = df_horizon4.merge(saha_aktivite_detay3[['yil', 'ay', 'grup_adi', 'ciro_kull_i̇ade_dus', 
                                                      'promosyon_tutari', 'i̇skonto_', 'en_guncel_kod']],
                                on=['en_guncel_kod', 'yil', 'ay', 'grup_adi'], how="left")

btt_aktivite = saha_aktivite_detay3[saha_aktivite_detay3["grup_adi"] == "GELENEKSEL KANAL"].reset_index(drop=True)
btt_aktivite["grup_adi"] = "BTT"

df_btt5 = df_btt4.merge(btt_aktivite[['yil', 'ay', 'grup_adi', 'ciro_kull_i̇ade_dus', 
                                      'promosyon_tutari', 'i̇skonto_', 'en_guncel_kod']],
                        on=['en_guncel_kod', 'yil', 'ay', 'grup_adi'], how="left")

In [143]:
df_horizon5.drop_duplicates(ignore_index=True, inplace=True)

In [144]:
df_btt5.drop_duplicates(ignore_index=True, inplace=True)

In [145]:
pasifik_df_sorted = df_pasifik5.copy()
horizon_saha_df_sorted = df_horizon5.copy()
btt_df_sorted = df_btt5.copy()

In [146]:
pas_backup = pasifik_df_sorted.copy()
hor_backup = horizon_saha_df_sorted.copy()
btt_backup = btt_df_sorted.copy()

---
# Kapsamı yeniden düzenleme
---

In [147]:
pasifik_df_sorted["indirim__"] = [0 if akt < 0 else akt for akt in pasifik_df_sorted["indirim_"]]
pasifik_df_sorted.drop("indirim_", axis=1, inplace=True)

horizon_saha_df_sorted["i̇skonto__"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in horizon_saha_df_sorted["i̇skonto_"]]
horizon_saha_df_sorted.drop("i̇skonto_", axis=1, inplace=True)

btt_df_sorted["i̇skonto__"] = [0 if ((akt >= 0.35) or (akt <=0.01)) else akt for akt in btt_df_sorted["i̇skonto_"]]
btt_df_sorted.drop("i̇skonto_", axis=1, inplace=True)

In [148]:
chng_cols = dict(zip(['Yıl', 'Ay', 'No_of_days', 'Weekdays_n', 'Weekdays_Ratio', 'Weekend_n',
                      'Weekend_Ratio', 'Actual_Holiday_n', 'Actual_Holiday_Ratio',
                      'Total_Holiday_n', 'Total_Holiday_Ratio', 'School_Day_n',
                      'School_Day_Ratio', 'School_Day_brdg_n', 'School_Day_brdg_Ratio',
                      'Ramadan_n', 'Ramadan_Ratio', 'Pandemic', 'Lockdown'],
                     
                     ["yil", "ay", "no_of_days", "weekdays_n", "weekdays_ratio", "weekend_n", 
                      "weekend_ratio", "actual_holiday_n", "actual_holiday_ratio",
                      "total_holiday_n", "total_holiday_ratio", "school_day_n", 
                      "school_day_ratio", "school_day_brdg_n", "school_day_brdg_ratio",
                      "ramadan_n", "ramadan_ratio", "pandemic", "lockdown"]))

In [149]:
calender_df.rename(columns=chng_cols, inplace=True)

In [150]:
pasifik_df_sorted["indirim__"].fillna(0, inplace=True)
horizon_saha_df_sorted["i̇skonto__"].fillna(0, inplace=True)
btt_df_sorted["i̇skonto__"].fillna(0, inplace=True)

In [151]:
horizon_saha_df_sorted.rename(columns={"i̇skonto__": "indirim__"}, inplace=True)
btt_df_sorted.rename(columns={"i̇skonto__": "indirim__"}, inplace=True)

In [152]:
pas_backup = pasifik_df_sorted.copy()
hor_backup = horizon_saha_df_sorted.copy()
btt_backup = btt_df_sorted.copy()

---
# Bir önceki aya yansımış aktiviteleri düzenleme
---

In [153]:
def aktivite_regulation(df):
    df_all = []
    for sku in df["en_guncel_kod"].unique():
        for grup in df["grup_adi"].unique():
            test = df[(df["en_guncel_kod"] == sku) & (df["grup_adi"] == grup)]
            for idx in test.index:
                if (idx-2 not in test.index) and (idx-1 in test.index): # bir öncekine bakacak. ilk ve sonraki satıra bakacak. -1 götür
                    if test.loc[idx-1, "koli"] > test.loc[idx, "koli"]:
                        test.loc[idx-1, "indirim__"] += test.loc[idx, "indirim__"]
                        test.loc[idx, "indirim__"] = 0
                elif (idx-1 not in test.index): # ilk satırdayız. pass
                    pass
                else:
                    dic = {}
                    dic.update({idx-2: test.loc[idx-2, "koli"], 
                                idx-1: test.loc[idx-1, "koli"],
                                idx: test.loc[idx, "koli"]})
                    max_idx = max(dic, key=dic.get)
                    if max_idx == idx:
                        pass
                    else:
                        test.loc[max_idx, "indirim__"] += test.loc[idx, "indirim__"]
                        test.loc[idx, "indirim__"] = 0
            df_all.append(test)
    df_all = pd.concat(df_all)
    return df_all

In [154]:
start_time = datetime.now()

In [155]:
horizon_saha_df_sorted, btt_df_sorted = aktivite_regulation(horizon_saha_df_sorted), aktivite_regulation(btt_df_sorted)

In [156]:
end_time = datetime.now()
print('Aktivite Regulation süresi: {}'.format(end_time - start_time))

Aktivite Regulation süresi: 0:12:41.945158


In [157]:
def pasifik_aktivite_regulation(df):
    df_all = []
    for sku in df["en_guncel_kod"].unique():
        for grup in df["grup_adi"].unique():
            test = df[(df["en_guncel_kod"] == sku) & (df["grup_adi"] == grup)]
            aktivite_tip_index = test[~test.aktivite_tipi.isna()].index.to_list()
            for idx in aktivite_tip_index:
                if idx == test.index[0]: # ilk satırsa atla
                    pass
                elif idx-1 in aktivite_tip_index: #bir önceki satırda aktivite varsa atla
                    pass
                else:
                    if test.loc[idx, 'koli'] < test.loc[idx-1, 'koli']: # adet sayısı bir önceki satırdan küçükse 
                        test.loc[idx-1, 'aktivite_tipi'] = test.loc[idx, 'aktivite_tipi'] # aktiviteyi bir önceki satıra yaz
                        test.loc[idx, 'aktivite_tipi'] = np.nan
                        aktivite_tip_index.remove(idx) #listeden remove et ki bir alt satırda varsa önceki var mı kontrolüne takılmasın
                        aktivite_tip_index.insert(0,0) #döngü listedeki elementlerin index'ine göre devam ettiği için en başa 0 insert et

            indirim_index = test[test.indirim__ != 0].index.to_list()
            for idx in indirim_index:
                if idx == test.index[0]: # ilk satır
                    pass 
                elif idx-1 in indirim_index: #bir önceki satırda indirim yüzdesi varsa atla
                    pass 
                else:
                    if test.loc[idx, 'koli'] < test.loc[idx-1, 'koli']:
                        test.loc[idx-1, 'indirim__'] = test.loc[idx, 'indirim__'] #bir üste taşındı
                        test.loc[idx, 'indirim__'] = 0  
                        indirim_index.remove(idx)  #listeden remove et ki bir alt satırda varsa önceki var mı kontrolüne takılmasın
                        indirim_index.insert(0, 0) #döngü listedeki elementlerin index'ine göre devam ettiği için en başa 0 insert et
            df_all.append(test)
    df_all = pd.concat(df_all)
    return df_all

In [158]:
start_time = datetime.now()

In [159]:
pasifik_df_sorted = pasifik_aktivite_regulation(pasifik_df_sorted)

In [160]:
end_time = datetime.now()
print('Pasifik Aktivite Regulation süresi: {}'.format(end_time - start_time))

Pasifik Aktivite Regulation süresi: 0:00:44.414135


In [161]:
pasifik_df_sorted.sort_values(by=["en_guncel_kod", "grup_adi", "date"], ignore_index=True, inplace=True)
horizon_saha_df_sorted.sort_values(by=["en_guncel_kod", "grup_adi", "date"], ignore_index=True, inplace=True)
btt_df_sorted.sort_values(by=["en_guncel_kod", "grup_adi", "date"], ignore_index=True, inplace=True)

btt_df_sorted.drop_duplicates(subset=["grup_adi", "en_guncel_kod", "date"], keep="first", ignore_index=True, inplace=True)
horizon_saha_df_sorted.drop_duplicates(subset=["date", "grup_adi", "en_guncel_kod"], keep="first", ignore_index=True, inplace=True)

In [162]:
horizon_saha_df_sorted["aktivite_tipi"] = np.nan
btt_df_sorted["aktivite_tipi"] = np.nan

In [163]:
pasifik_df_sorted.rename(columns={'i̇ndirimli_raf_satis_fiyati': 'ciro_kull_i̇ade_dus',
                                  'raf_tavsiye_satis_fiyati': 'promosyon_tutari'}, inplace=True)

In [164]:
pasifik_df_sorted["Kanal"] = "pasifik"
horizon_saha_df_sorted["Kanal"] = "horizon"
btt_df_sorted["Kanal"] = "btt"

In [165]:
horizon_df_eski = [] 
for docs_ in btt_lst:
    horizon_df_eski.append(pd.read_excel(params_["path"]["horizon_koli_path"]+docs_, skiprows=1, sheet_name="Horizon Saha Satış", usecols="B:N").rename(columns=chng_cols_beginning))

In [166]:
horizon_df_eski = pd.concat(horizon_df_eski, ignore_index=True)
hor_kat_marka = horizon_df_eski.drop_duplicates(subset=["Kategori Adı", "Marka Adı", "Ürün Kodu"], ignore_index=True)[["Kategori Adı", "Marka Adı", "Ürün Kodu"]]
hor_kat_marka.rename(columns={"Kategori Adı": "kategori_adi", "Marka Adı": "marka_adi", "Ürün Kodu": "en_guncel_kod"}, inplace=True)

In [167]:
horizon_saha_df_sorted = horizon_saha_df_sorted.merge(hor_kat_marka, on="en_guncel_kod", how="left")

---
# Pasifik Marka İsmi Düzenleme
---

In [168]:
pasifik_marka = pasifik_df_all[["Grup Adı", "En Güncel Kod", "Marka Adı"]].drop_duplicates(subset=["Grup Adı", "En Güncel Kod", "Marka Adı"], ignore_index=True)
pasifik_marka.rename(columns={"Grup Adı": "grup_adi", "En Güncel Kod": "en_guncel_kod", "Marka Adı": "marka_adi"}, inplace=True)
pasifik_marka.grup_adi = pasifik_marka.grup_adi.apply(lambda x: "Diğer_Pasifik" if x=="Diğer" else x)

In [169]:
pasifik_df_sorted = pasifik_df_sorted.merge(pasifik_marka, how="left")

---
# Horizon Marka İsmi Düzenleme
---

In [170]:
horizon_marka = horizon_saha_df_all[["Grup Adı", "En Güncel Kod", "Marka Adı"]].drop_duplicates(subset=["Grup Adı", "En Güncel Kod", "Marka Adı"], ignore_index=True)
horizon_marka.rename(columns={"Grup Adı": "grup_adi", "En Güncel Kod": "en_guncel_kod", "Marka Adı": "marka_adi"}, inplace=True)
horizon_marka.grup_adi = horizon_marka.grup_adi.apply(lambda x: "Diğer_Horizon" if x=="Diğer" else x)

In [171]:
horizon_marka["marka_adi"] = horizon_marka["marka_adi"].apply(lambda x: "ALTINBAŞAK" if x == "ÜLKER GRİSSİNİ" else x)

In [172]:
try:
    horizon_saha_df_sorted.drop(columns=["marka_adi"], axis=1, inplace=True)
except KeyError:
    pass

In [173]:
horizon_saha_df_sorted = horizon_saha_df_sorted.merge(horizon_marka, how="left")
horizon_saha_df_sorted.drop_duplicates(inplace=True, ignore_index=True)

---
# BTT Marka İsmi Düzenleme
---

In [174]:
btt_marka = btt_df_all[["Grup Adı", "En Güncel Kod", "Marka Adı"]].drop_duplicates(subset=["Grup Adı", "En Güncel Kod", "Marka Adı"], ignore_index=True)
btt_marka.rename(columns={"Grup Adı": "grup_adi", "En Güncel Kod": "en_guncel_kod", "Marka Adı": "marka_adi"}, inplace=True)
btt_marka.grup_adi = btt_marka.grup_adi.apply(lambda x: "Diğer_Horizon" if x=="Diğer" else x)

In [175]:
btt_marka["marka_adi"] = btt_marka["marka_adi"].apply(lambda x: "ALTINBAŞAK" if x == "ÜLKER GRİSSİNİ" else x)

In [176]:
try:
    btt_df_sorted.drop(columns=["marka_adi"], axis=1, inplace=True)
except KeyError as e:
    print(e)
    pass

"['marka_adi'] not found in axis"


In [177]:
btt_df_sorted = btt_df_sorted.merge(btt_marka, how="left")
btt_df_sorted.drop_duplicates(inplace=True, ignore_index=True)

In [178]:
df_all = pd.concat([pasifik_df_sorted, horizon_saha_df_sorted, btt_df_sorted], axis=0, ignore_index=True)
df_all.drop(columns="kanal", axis=1, inplace=True)

In [179]:
df_all = df_all[["yil", "ay", "Kanal"]+df_all.drop(columns=["yil", "ay", "Kanal"]).columns.to_list()]

In [180]:
df_all.aktivite_tipi.fillna("Yok", inplace=True)
try: df_all.drop(columns=["durum", "promosyon_tutari", "ciro_kull_i̇ade_dus"], axis=1, inplace=True)
except: pass

In [181]:
df_all["koli_new"] = df_all["koli"]
df_all["koli_new"] = np.where(df_all["koli_new"] == 0, np.nan, df_all["koli_new"])

In [182]:
df_all = df_all[df_all["portfoy"] == 1].reset_index(drop=True)

---
# Geçmiş ayları çalıştırmak için bu kısmı kullanıyoruz
---

In [183]:
df_all = df_all[~(df_all["en_guncel_kod"].isin(["Delist", " Delist", " Delist ", "Delist ", "delist", " delist", " delist ", "delist "]))].reset_index(drop=True)

In [184]:
kanal_kat_unique = df_all.drop_duplicates(subset=["Kanal", "kategori_adi"], ignore_index=True)[["Kanal", "kategori_adi"]]

In [185]:
df_all["back_to_school"] = df_all["ay"].apply(lambda x: 1 if x in [9, 2] else 0)

In [186]:
kanal_kat_ = []
corr_ = []
dt_feature_importance_ = []
xgb_feature_importance_ = []
rf_feature_importance_ = []

for idx in kanal_kat_unique.index:
    tmp = df_all[(df_all["Kanal"] == kanal_kat_unique["Kanal"][idx]) & (df_all["kategori_adi"] == kanal_kat_unique["kategori_adi"][idx])]
    tmp.fiyat.fillna(0, inplace=True)
    tmp_corr = tmp[["koli", "no_of_days", 'back_to_school', 'weekdays_ratio', 'weekend_ratio',
                    'total_holiday_ratio', 'school_day_ratio', 'ramadan_ratio', 'indirim__', 'fiyat']].corr()
    tmp_dt = tmp[["koli", "no_of_days", 'back_to_school', 'weekdays_ratio', 'weekend_ratio',
                  'total_holiday_ratio', 'school_day_ratio', 'ramadan_ratio', 'indirim__', 'fiyat']]
    X = tmp_dt.drop("koli", axis=1)
    y = tmp_dt.koli
    dt = DecisionTreeRegressor().fit(X, y)
    xgb = XGBRegressor().fit(X, y)
    rf = RandomForestRegressor().fit(X, y)
    
    dt_importance = dt.feature_importances_
    xgb_importance = xgb.feature_importances_
    rf_importance = rf.feature_importances_
    dt_feat_imprtnce = {}
    xgb_feat_imprtnce = {}
    rf_feat_imprtnce = {}
    for i, v in enumerate(dt_importance):
        dt_feat_imprtnce.update({X.columns[i]: v})
    for i, v in enumerate(xgb_importance):
        xgb_feat_imprtnce.update({X.columns[i]: v})
    for i, v in enumerate(rf_importance):
        rf_feat_imprtnce.update({X.columns[i]: v})
    dt_feat_imprtnce = {k: v for k, v in sorted(dt_feat_imprtnce.items(), key=lambda item: item[1], reverse=True)}
    xgb_feat_imprtnce = {k: v for k, v in sorted(xgb_feat_imprtnce.items(), key=lambda item: item[1], reverse=True)}
    rf_feat_imprtnce = {k: v for k, v in sorted(rf_feat_imprtnce.items(), key=lambda item: item[1], reverse=True)}
    kanal_kat_.append(kanal_kat_unique.loc[idx, "Kanal"] + "_" + kanal_kat_unique.loc[idx, "kategori_adi"])
    corr_.append(tmp_corr)
    dt_feature_importance_.append(dt_feat_imprtnce)
    xgb_feature_importance_.append(xgb_feat_imprtnce)
    rf_feature_importance_.append(rf_feat_imprtnce)

In [187]:
summary_ = pd.DataFrame({"hierarchy": kanal_kat_,
                         "correlation": corr_,
                         "dt_fi": dt_feature_importance_,
                         "xgb_fi": xgb_feature_importance_,
                         "rf_fi": rf_feature_importance_})

In [188]:
df_all_backup = df_all.copy()

---
# Unique Fiyatların düzenlenmesi. Bazı kısımlarda na geldi çünkü önceki tarihlerin fiyatları yok. Data 2017 Ocak'ta başlıyor ama fiyat datası 2017 Aralıkta başlıyor. Bu durumdan dolayı missing imputation yaptım.
---

In [189]:
fiyat_unique.rename(columns={"kanal": "Kanal"}, inplace=True)

In [190]:
df_all_v2 = df_all.copy()
df_all_v2.en_guncel_kod = df_all_v2.en_guncel_kod.apply(lambda x: int(x))

In [191]:
fiyat_unique = fiyat_unique[~(fiyat_unique["en_guncel_kod"].isin(["Delist", " Delist", " Delist ", "Delist ", "delist", " delist", " delist ", "delist "]))].reset_index(drop=True)

In [192]:
fiyat_unique.en_guncel_kod = fiyat_unique.en_guncel_kod.apply(lambda x: int(x))

In [193]:
try:
    df_all_v2.drop(columns=["fiyat", "fiyat_gecisi"], axis=1, inplace=True)
except KeyError as e:
    print(e)

In [194]:
df_all_v2 = df_all_v2.merge(fiyat_unique, how="left", on=["date", "en_guncel_kod", "Kanal"])

In [195]:
null_fiyat_ = df_all_v2[df_all_v2["fiyat"].isna()][["grup_adi", "en_guncel_kod"]].drop_duplicates(subset=["grup_adi", "en_guncel_kod"], ignore_index=True)

In [196]:
df_all_v3 = df_all_v2.copy()
df_all_v3.fiyat.fillna(method="bfill", inplace=True)
df_all_v3.fiyat.fillna(0, inplace=True)
df_all_v3.fiyat_gecisi.fillna(method="bfill", inplace=True)
df_all_v3.fiyat_gecisi.fillna(0, inplace=True)

---
# Koli Missing Imputation (Mean)
---

In [197]:
def missing_imputation(df__, unique_grup, unique_sku, loop):
    print("Başla ------>", unique_grup[loop], unique_sku[loop])
    tmp = df__[(df__["grup_adi"] == unique_grup[loop]) & (df__["en_guncel_kod"] == unique_sku[loop])]
    if tmp["koli_new"].isna().sum() > 0:
        mean = tmp["koli_new"].mean()
        tmp.koli_new.fillna(mean, inplace=True)
    else: pass
    print("Bitti ------>", unique_grup[loop], unique_sku[loop])
    return tmp

In [198]:
unique_list = df_all_v3[["grup_adi", "en_guncel_kod"]].drop_duplicates(ignore_index=True)
unique_grup = unique_list["grup_adi"].to_list()
unique_sku = unique_list["en_guncel_kod"].to_list()

In [199]:
start_time = datetime.now()

In [200]:
df_all_v4 = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    df__ = df_all_v3.copy()
    func = partial(missing_imputation, df__, unique_grup, unique_sku)
    #func = partial(missing_imputation_v2, df__, unique_grup, unique_sku)
    loop = list(unique_list.index)
    with mp.Pool(available_cpu) as p:
        df_all_v4.append(p.map(func, loop))

In [201]:
end_time = datetime.now()
print('Missing Imputation: {}'.format(end_time - start_time))

Missing Imputation: 0:00:32.818623


In [202]:
df_all_v4 = pd.concat(df_all_v4[0], ignore_index=True)

In [203]:
df_all_v4["koli_new"].fillna(0, inplace=True)

---
# Trend Seasonality Decomposition with parallel processsing
---

In [204]:
unique_list = df_all_v4.drop_duplicates(subset=["grup_adi", "en_guncel_kod"], ignore_index=True)[["grup_adi", "en_guncel_kod"]]
unique_grup = unique_list["grup_adi"].to_list()
unique_en_guncel_kod = unique_list["en_guncel_kod"].to_list()

In [205]:
df_all_v4.drop_duplicates(ignore_index=True, inplace=True)

In [206]:
def trend_seasonality_decomp(df__, unique_grup, unique_en_guncel_kod, loop):
    from statsmodels.tsa.seasonal import STL
    import numpy as np
    print("Başladı ---------->", unique_grup[loop], unique_en_guncel_kod[loop])
    temp_df = df__[(df__["en_guncel_kod"] == unique_en_guncel_kod[loop]) & 
                   (df__["grup_adi"] == unique_grup[loop])]
    if len(temp_df) > 2:
        df_ts = temp_df[['koli_new','date']]
        df_ts.set_index('date',inplace=True)

        result = STL(df_ts).fit()
        temp_df['season'] = list(result.seasonal)
        temp_df['trend']  = list(result.trend)
        temp_df['residual']  = list(result.resid)
    else:
        temp_df['season'] = np.nan
        temp_df['trend']  = np.nan
        temp_df['residual']  = np.nan
    print("Bitti ---------->", unique_grup[loop], unique_en_guncel_kod[loop])
    return temp_df

In [207]:
start_time = datetime.now()

In [208]:
df_all_v5 = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    df__ = df_all_v4.copy()
    func = partial(trend_seasonality_decomp, df__, unique_grup, unique_en_guncel_kod)
    loop = list(unique_list.index)
    with mp.Pool(available_cpu) as p:
        df_all_v5.append(p.map(func, loop))

In [209]:
end_time = datetime.now()
print('Dataların yüklenmesi: {}'.format(end_time - start_time))

Dataların yüklenmesi: 0:00:39.316086


In [210]:
df_all_v5 = pd.concat(df_all_v5[0], ignore_index=True)

In [211]:
fiyat_next_four = df_all_v5[["grup_adi", "en_guncel_kod", "fiyat", "date"]]
fiyat_next_four.sort_values(by=["grup_adi", "en_guncel_kod", "date"], ascending=[True, True, False], inplace=True, ignore_index=True)
fiyat_next_four = fiyat_next_four.groupby(["grup_adi", "en_guncel_kod"]).agg({"date": "max", "fiyat": "max"}).reset_index()
fiyat_next_four = fiyat_next_four[["grup_adi", "en_guncel_kod", "fiyat"]].drop_duplicates(ignore_index=True)

---
# Scope Belirleme (24 aylık veri varsa time series çalışsın. Yoksa moving average)
---

In [212]:
def scope(df__, unique_grup, unique_sku, threshold, loop):
    print("Başla ----->", loop)
    tmp = df__[(df__["grup_adi"] == unique_grup[loop]) & (df__["en_guncel_kod"] == unique_sku[loop])]
    if len(tmp[tmp["koli"] != 0]) >= 24:
        tmp["scope"] = "time_series"
    else:
        tmp["scope"] = "moving_average"
    print("Bitti ----->", loop)
    return tmp

In [213]:
unique_list = df_all_v5[["grup_adi", "en_guncel_kod"]].drop_duplicates(ignore_index=True)
unique_grup = unique_list["grup_adi"].to_list()
unique_sku = unique_list["en_guncel_kod"].to_list()

In [214]:
start_time = datetime.now()

In [215]:
df_all_v6 = []

if __name__ == "__main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    df__ = df_all_v5.copy()
    func = partial(scope, df__, unique_grup, unique_sku, 24)
    loop = list(unique_list.index)
    with mp.Pool(available_cpu) as p:
        df_all_v6.append(p.map(func, loop))

In [216]:
end_time = datetime.now()
print('Dataların yüklenmesi: {}'.format(end_time - start_time))

Dataların yüklenmesi: 0:00:32.519044


In [217]:
df_all_v6 = pd.concat(df_all_v6[0], ignore_index=True)

In [218]:
kapsam_all.columns = change_col_name(kapsam_all)
eslenik_kod_df.columns = change_col_name(eslenik_kod_df)

In [219]:
kapsam_all["urun_kodu"] = kapsam_all["urun_kodu"].apply(lambda x: int(x.split("-")[0]+x.split("-")[1]))
kapsam_all = kapsam_all.merge(eslenik_kod_df[["urun_kodu", "en_guncel_kod"]], how="left", on="urun_kodu")
kapsam_all["en_guncel_kod"].fillna(kapsam_all["urun_kodu"], inplace=True)
kapsam_all.drop_duplicates(subset=["en_guncel_kod", "grup_adi"], inplace=True, ignore_index=True)
kapsam_all = kapsam_all[kapsam_all["en_guncel_kod"] != "delist"].reset_index(drop=True)

In [220]:
pasifik_df_sorted = df_all_v6[df_all_v6["Kanal"] == "pasifik"].reset_index(drop=True)
df_all_v6 = df_all_v6[df_all_v6["Kanal"] != "pasifik"].reset_index(drop=True)
pasifik_df_sorted = pasifik_df_sorted.merge(kapsam_all[["en_guncel_kod", "grup_adi", "durum"]], how="left", on=["en_guncel_kod", "grup_adi"])

In [221]:
idx_to_write_deneme = list(pasifik_df_sorted[(pasifik_df_sorted["durum"].isna()) & (pasifik_df_sorted["grup_adi"].isin(["A101", "ŞOK", "BİM"]))].index)
check = pasifik_df_sorted[(pasifik_df_sorted["grup_adi"] == "BİM") & (pasifik_df_sorted["portfoy"] == 1)]
check = check[["en_guncel_kod", "grup_adi", "portfoy", "durum"]].reset_index(drop=True)

In [222]:
pasifik_df_sorted.loc[idx_to_write_deneme, "durum"] = "DENEME"
df_all_v6["durum"] = np.nan

In [223]:
pasifik_df_backup_kapsam = pasifik_df_sorted.copy()

In [224]:
change_scope_index = pasifik_df_sorted[(pasifik_df_sorted["portfoy"] == 1) & 
                                       (pasifik_df_sorted["Kanal"] == "pasifik") & 
                                       (pasifik_df_sorted["durum"].isin(['DENEME+BÖLGESEL SATIŞ', 'DENEME'])) & 
                                       (~pasifik_df_sorted["grup_adi"].isin(["Diğer_Pasifik", "MİGROS"]))].index
pasifik_df_sorted.loc[change_scope_index, "scope"] = "kapsam_disi"

In [225]:
listeli_olmayan_sku = pasifik_df_sorted[(pasifik_df_sorted["grup_adi"].isin(["ŞOK", "A101", "BİM"])) & 
                                        (~(pasifik_df_sorted["durum"].isin(["Listeli", "LİSTELİ"]))) & 
                                        (pasifik_df_sorted["portfoy"] == 1)]

In [226]:
listeli_olmayan_sku = listeli_olmayan_sku[["grup_adi", "en_guncel_kod"]]
listeli_olmayan_sku.drop_duplicates(inplace=True, ignore_index=False)

In [227]:
df_all_v6 = pd.concat([pasifik_df_sorted, df_all_v6], ignore_index=False)

---
# Historic datanın SAS'a yüklendiği kısım
---

In [228]:
df_sas_upload = df_all_v6.copy()
df_sas_upload_date_normal = df_sas_upload.copy()

In [ ]:
try: 
    conn.userinfo() 
except: 
    print("SWAT connections was never created or you lost the connection. Trying to create connection...") 
    access_ = True 
    while access_: 
        try: 
            conn = swat.CAS('yhtrcl-sasccnt1.yildiz.domain', 5570, username=params_["login_info"]["username"], password=params_["login_info"]["password"]) 
            access_ = False 
            print("Accessed!") 
        except:
            print("Got error. Trying to reconnect...") 
            continue

df_sas_upload["date"] = df_sas_upload["date"].apply(lambda x: (x - datetime(1960, 1, 1)).days) 
#ds_table = conn.CASTable('data_'+data_name_dict[params_["time_info_for_debugging"]["ay"]]+"_koli", caslib=params_['caslib_info']['caslib_name'], replace=True) 
#ds_table.table.dropTable(quiet=True) 
if conn.table.tableExists(name='data_'+data_name_dict[params_["time_info_for_debugging"]["ay"]]+"_koli",caslib=params_['caslib_info']['caslib_name']).exists:
    conn.table.dropTable(name='data_'+data_name_dict[params_["time_info_for_debugging"]["ay"]]+"_koli",caslib=params_['caslib_info']['caslib_name'])
conn.upload(data=df_sas_upload, casout={'caslib':params_['caslib_info']['caslib_name'], 'name':'data_'+data_name_dict[params_["time_info_for_debugging"]["ay"]]+"_koli", 'promote':True},  
            importoptions={'vars':{'date':{'format': 'MMDDYY', "type": "double", "length": 10}}})

In [229]:
df_sas_upload_date_normal["datanin_kesildigi_tarih"] = datetime(params_["time_info_for_debugging"]["yil"], params_["time_info_for_debugging"]["ay"], 1)

In [291]:
if __name__ == '__main__':
    
#    create_connection(r"../data/db/"+params_["db_info"]["db_name"]+".sqlite") # creates new database if there is no db.
#    create_new_table(all_final_preds11, params_["db_info"]["db_name"], params_["db_info"]["current_results_table"]) # creates new table if there is no table.
#    upload_new_results(all_final_preds11, params_["db_info"]["db_name"], params_["db_info"]["current_results_table"]) # drop previous predictions and uploads new predictions for current period.
    
    update_table(df_sas_upload_date_normal, params_["db_info"]["db_name"], params_["db_info"]["historic_data"]) # inserts to historic database.

New data is inserted into DS_HISTORY_DATA table.


---
---
---
---
---
---
---
# ÖNCE SASTAKİ SONUÇLARIN ÇALIŞMASI GEREKİYOR. SONRA AŞAĞIYI ÇALIŞTIRMAYA BAŞLA!
---
---
---
---
---
---
---

# SAS'tan sonuçları çekme işlemi
---

In [230]:
try: 
    conn.userinfo() 
except: 
    print("SWAT connections was never created or you lost the connection. Trying to create connection...") 
    access_ = True 
    while access_: 
        try: 
            conn = swat.CAS('yhtrcl-sasccnt1.yildiz.domain', 5570, username=params_["login_info"]["username"], password=params_["login_info"]["password"]) 
            access_ = False 
            print("Accessed!")
            outfor = pd.DataFrame(conn.CASTable(caslib=params_["caslib_info"]["caslib_name"],name=params_["sas_results"]["result_table_name"]).to_frame())
#            outfor = outfor[["Kanal", "grup_adi", "urun_adi", "date", "ACTUAL", "PREDICT"]]
#            outfor["date"] = pd.to_datetime(outfor["date"], format="%Y-%m-%d", errors="coerce")
        except:
            print("Got error. Trying to reconnect...") 
            continue

SWAT connections was never created or you lost the connection. Trying to create connection...
Accessed!


# Sonuçları notebook hata çıkarırsa diye kaydediyorum
---

In [231]:
outfor.date.unique()

array([datetime.date(2022, 6, 1), datetime.date(2022, 5, 1),
       datetime.date(2022, 3, 1), datetime.date(2022, 4, 1)], dtype=object)

In [232]:
outfor.to_csv(f'sas_results_{data_name_dict[ay_threshold+1]}', index=False)

In [234]:
indirim_dict = {
    
    0: "0",
    1: "%0 - %1",
    2: "%1 - %2",
    3: "%2 - %3",
    4: "%3 - %4",
    5: "%4 - %5",
    6: "%5 - %6",
    7: "%6 - %7",
    8: "%7 - %8",
    9: "%8 - %9",
    10: "%9 - %10",
    11: "%10 - %15",
    12: "%15 - max"
}

indirim_dict2 = {
    
    0: '0',
    1: '%0 - %1',
    2: '%1 - %2',
    3: '%2 - %3',
    4: '%3 - %4',
    5: '%4 - %5',
    6: '%5 - %6',
    7: '%6 - %7',
    8: '%7 - %8',
    9: '%8 - %9',
    10: '%9 - %10',
    12.5: '%10 - %15',
    15: '%15 - max'}

# Bu aşağıdaki sütunlar DWH'a yüklememiz için olması gerekiyor. Gerçekleşenler olmadığı için 0 yazıyorum. Tahmin sapması için de 0 diyorum çünkü gelecek ayların gerçekleşenlerini bilmiyoruz.
---

In [235]:
outfor["gerceklesen_satis_adedi"] = 0
outfor["gerceklesen_koli"] = 0
outfor["tahmin_sapmasi"] = 0

In [236]:
outfor["en_guncel_kod"] = outfor["en_guncel_kod"].astype("int64")

In [237]:
grup_sku_unique = outfor[["en_guncel_kod", "grup_adi"]].drop_duplicates(ignore_index=True)

In [238]:
outfor2 = outfor.copy()
outfor2["winner_model"] = outfor2["winner_model"].apply(lambda x: "autoforecast" if x == "AF" else(
                                                                    'moving_average' if x == 'MA3' else(
                                                                        'hierarchical_forecast' if x.startswith('HF') else x)))

In [239]:
outfor2.winner_model.unique()

array(['', 'DIAG1_UCM1', 'DIAG1_REGARIMA1', 'DIAG1_ARIMAX1', 'naiveModel',
       'ESMBEST', 'DIAG1_ESM1', 'DIAG1_IDM1'], dtype=object)

In [240]:
outfor2 = outfor2[['en_guncel_kod', 'grup_adi', 'winner_prediction', 'min_mape', 'winner_model', 'date', 
                   'aktivite_tipi', 'indirim_yuzdesi','gerceklesen_satis_adedi','gerceklesen_koli','tahmin_sapmasi']]

# Grup adı, ana kategori, kategori, marka isimlerini left join ile getiriyoruz.
---

In [241]:
kanal_ana_kat_kat_marka_list = df_all_v6[["Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi", "en_guncel_kod"]].drop_duplicates(ignore_index=True)
outfor3 = outfor2.merge(kanal_ana_kat_kat_marka_list, how="left")

In [242]:
outfor2.shape, outfor3.shape

((209924, 11), (209924, 15))

In [243]:
grup_kanal_dict = {
    
    'A101': 'pasifik', 
    'BİM': 'pasifik', 
    'Diğer_Pasifik': 'pasifik', 
    'MİGROS': 'pasifik', 
    'ŞOK': 'pasifik', 
    'Diğer_Horizon': 'horizon', 
    'GELENEKSEL KANAL': 'horizon', 
    'ORTA MARKET': 'horizon', 
    'POTANSİYEL MARKET': 'horizon', 
    'YEREL ZİNCİR': 'horizon', 
    'BTT': 'btt'
                    
                }

---
# Listeli Olmayan Sonuçların Getirilmesi
---

In [244]:
listeli_olmayan_df = listeli_olmayan_sku.copy()
listeli_olmayan_df["Kanal"] = listeli_olmayan_df["grup_adi"].map(grup_kanal_dict)

In [245]:
def create_comb_listeli_olmayan(excels):
#    month_ = datetime.now().month
#    year_ = datetime.now().year
    month_ = params_["time_info_for_debugging"]["ay"]
    year_ = params_["time_info_for_debugging"]["yil"]
    pas_s=list(itertools.product([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12.5, 15],['Yok', 'Mağaza içi/Dağılım', 'İn&out', 'Mutluluk', 'Çoklu Alım', 'Kasiyer', 'CRM']))
    diger_s=list(itertools.product([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12.5, 15],["Yok"]))
    df_all__=[]
    for i in excels.index:
        tmp_=pd.DataFrame(excels.loc[i]).T
        new_date = datetime(year_, month_, 1)
        if tmp_['Kanal'].values[0]=='pasifik':
            for trh in range(1, 5):
                new_date_ = new_date + relativedelta(months=trh)
                create_new = pd.DataFrame(tmp_.iloc[-1])
                create_new.loc["tarih"] = datetime(new_date_.year, new_date_.month, 1)
                tmp = pd.concat([create_new.T]*len(pas_s), ignore_index=True)
                for j in range(len(pas_s)):
                    tmp.loc[j,'indirim_yuzdesi'] = pas_s[j][0]
                    tmp.loc[j,'aktivite_tipi'] = pas_s[j][1]
                df_all__.append(tmp)
        else:
            for trh in range(1, 5):
                new_date_ = new_date + relativedelta(months=trh)
                create_new = pd.DataFrame(tmp_.iloc[-1])
                create_new.loc["tarih"] = datetime(new_date_.year, new_date_.month, 1)
                tmp = pd.concat([create_new.T]*len(diger_s), ignore_index=True)
                for k in range(len(diger_s)):
                    tmp.loc[k,'indirim_yuzdesi'] = diger_s[k][0]
                    tmp.loc[k,'aktivite_tipi'] = diger_s[k][1]
                df_all__.append(tmp)
    df_all__ = pd.concat(df_all__, ignore_index=True)
    return df_all__

In [246]:
listeli_olmayan_df2 = create_comb_listeli_olmayan(listeli_olmayan_df)

In [247]:
listeli_olmayan_df3 = listeli_olmayan_df2.copy()
listeli_olmayan_sku_list = listeli_olmayan_df2[~(listeli_olmayan_df2["aktivite_tipi"] == "Yok")]
listeli_olmayan_df3["pred"] = np.nan

In [248]:
t_2_date = df_all_v6.date.max() + relativedelta(months=2)
for idx in listeli_olmayan_sku_list.index:
    tmp_hist = df_all[(df_all["grup_adi"] == listeli_olmayan_sku_list.loc[idx, "grup_adi"]) & 
                      (df_all["en_guncel_kod"] == listeli_olmayan_sku_list.loc[idx, "en_guncel_kod"]) & 
                      (df_all["aktivite_tipi"] == listeli_olmayan_sku_list.loc[idx, "aktivite_tipi"])]
    if len(tmp_hist) > 0:
        mean_ = tmp_hist.koli.mean()
        keep_index = listeli_olmayan_df3[(listeli_olmayan_df3["grup_adi"] == listeli_olmayan_sku_list.loc[idx, "grup_adi"]) & 
                                         (listeli_olmayan_df3["en_guncel_kod"] == listeli_olmayan_sku_list.loc[idx, "en_guncel_kod"]) & 
                                         (listeli_olmayan_df3["aktivite_tipi"] == listeli_olmayan_sku_list.loc[idx, "aktivite_tipi"]) & 
                                         (listeli_olmayan_df3["tarih"] == t_2_date)].index

        listeli_olmayan_df3.loc[keep_index, "pred"] = mean_
    else: pass

In [249]:
listeli_olmayan_df3.pred.fillna(0, inplace=True)

In [250]:
listeli_olmayan_df3["del_col"] = listeli_olmayan_df3["grup_adi"].astype(str) + "_" + listeli_olmayan_df3["en_guncel_kod"].astype(str)
listeli_olmayan_df3["akt_ind_check"] = listeli_olmayan_df3["aktivite_tipi"].astype(str) + "_" + listeli_olmayan_df3["indirim_yuzdesi"].astype(str)
listeli_olmayan_df3["type"] = "listeli_olmayan_sku"

In [251]:
listeli_olmayan_df3.rename(columns={"tarih": "date"}, inplace=True)

In [252]:
listeli_olmayan_df3["indirim_yuzdesi"] = listeli_olmayan_df3["indirim_yuzdesi"].map(indirim_dict2)
listeli_olmayan_df3.drop(columns=["del_col", "akt_ind_check"], axis=1, inplace=True)

# sas sonuçlarından işimize yaramayan sütunları çıkartıyoruz
---

In [253]:
listeli_olmayan_df3.rename(columns={"pred": "tahmin_koli", "type": "tahmin_edilme_yontemi"}, inplace=True)

In [254]:
listeli_olmayan_df3["datanin_etiketlendigi_algorit"] = listeli_olmayan_df3["tahmin_edilme_yontemi"]
listeli_olmayan_df3["gerceklesen_koli"] = 0
listeli_olmayan_df3["gerceklesen_satis_adedi"] = 0
listeli_olmayan_df3["mape"] = 0
listeli_olmayan_df3["tahmin_sapmasi"] = 0

In [255]:
listeli_olmayan_df3 = listeli_olmayan_df3.merge(kanal_ana_kat_kat_marka_list, how="left")
#listeli_olmayan_df3 = listeli_olmayan_df3[outfor3.columns.to_list()]
listeli_olmayan_df3 = listeli_olmayan_df3[['en_guncel_kod','grup_adi','tahmin_koli','tahmin_sapmasi','tahmin_edilme_yontemi','date',
                                           'Kanal','ana_kategori_adi','kategori_adi','marka_adi','indirim_yuzdesi','aktivite_tipi',
                                           'mape','gerceklesen_koli','gerceklesen_satis_adedi']]
#Alttaki cell üste alınıp bu cell'deki 2. satır açılıp 3. satır silinebilir

In [256]:
#outfor3.drop(columns=["indirim", "aktivite", "dolu_gozlem", "toplam_gozlem", "autoforecast_mape", "ma3_mape", "autoforecast_predict", "ma3_predict"], axis=1, inplace=True)
outfor3.rename(columns={"winner_prediction": "tahmin_koli", "min_mape": "mape", "winner_model": "tahmin_edilme_yontemi"}, inplace=True)

In [257]:
outfor3.head()

,en_guncel_kod,grup_adi,tahmin_koli,mape,tahmin_edilme_yontemi,date,aktivite_tipi,indirim_yuzdesi,gerceklesen_satis_adedi,gerceklesen_koli,tahmin_sapmasi,Kanal,ana_kategori_adi,kategori_adi,marka_adi
0,103304,A101,11574.5,7.212388,,2022-06-01,CRM,%2 - %3,0,0,0,pasifik,ATIŞTIRMALIK,KEK,DANKEK
1,103304,A101,11574.5,7.212388,,2022-06-01,Yok,%1 - %2,0,0,0,pasifik,ATIŞTIRMALIK,KEK,DANKEK
2,103304,A101,11574.5,7.212388,,2022-06-01,Mutluluk,%1 - %2,0,0,0,pasifik,ATIŞTIRMALIK,KEK,DANKEK
3,103304,A101,11574.5,7.212388,,2022-06-01,Çoklu Alım,%2 - %3,0,0,0,pasifik,ATIŞTIRMALIK,KEK,DANKEK
4,103304,A101,11574.5,7.212388,,2022-05-01,Kasiyer,%4 - %5,0,0,0,pasifik,ATIŞTIRMALIK,KEK,DANKEK


In [258]:
outfor4 = outfor3.copy()

In [259]:
listeli_olmayan_sku["grup_sku"] = listeli_olmayan_sku["grup_adi"] + "_" + listeli_olmayan_sku["en_guncel_kod"].astype(str)

# Listeli olmayan ürünlerin sonuçları SAS'ta da bulunuyor. O yüzden iki kısımda da prediction'ı var gibi bir durum söz konusu. SAS sonuçlarından listeli olmaynaları çıkartıyoruz.
---

In [260]:
outfor5 = []
check_list = listeli_olmayan_sku["grup_sku"].to_list()
for index in grup_sku_unique.index:
    tmp = outfor4[(outfor4["grup_adi"] == grup_sku_unique.loc[index, "grup_adi"]) & 
                  (outfor4["en_guncel_kod"] == grup_sku_unique.loc[index, "en_guncel_kod"])]
    if tmp.iloc[0]["grup_adi"] + "_" + str(tmp.iloc[0]["en_guncel_kod"]) in check_list:
        pass
    else:
        outfor5.append(tmp)

In [261]:
outfor5 = pd.concat(outfor5, ignore_index=True)

---
# TÜM BİRLEŞTİRİLMİŞ SONUÇLAR
---

In [262]:
outfor5_backup = outfor5.copy()

In [263]:
outfor5.date = pd.to_datetime(outfor5.date)

In [264]:
all_final_preds6 = pd.concat([outfor5, listeli_olmayan_df3], ignore_index=True, axis=0)

In [265]:
check__2 = all_final_preds6.groupby(["date", "Kanal", "grup_adi", "en_guncel_kod"]).count().reset_index()

In [266]:
urun_isimleri = eslenik_kod_df[["urun_kodu", "urun_adi_orjinal"]]
urun_isimleri.rename(columns={"urun_kodu": "en_guncel_kod", "urun_adi_orjinal": "urun_adi"}, inplace=True)

In [267]:
col_name_dwh = {
    
    "date": "TARIH",
    "Kanal": "KANAL",
    "grup_adi": "GRUP_ADI",
    "ana_kategori_adi": "ANA_KATEGORI_ADI",
    "kategori_adi": "KATEGORI_ADI",
    "marka_adi": "MARKA_ADI",
    "urun_adi": "URUN_ADI",
    "en_guncel_kod": "EN_GUNCEL_KOD",
    "gerceklesen_satis_adedi": "GERCEKLESEN_SATIS_ADEDI",
    "tahmin_adet": "TAHMIN_ADET",
    "tahmin_ust_sinir": "TAHMIN_UST_SINIR",
    "tahmin_alt_sinir": "TAHMIN_ALT_SINIR",
    "koli_ici_adet": "KOLI_ICI_ADET",
    "tahmin_koli": "TAHMIN_KOLI",
    "mape": "MAPE",
    "gerceklesen_koli": "GERCEKLESEN_KOLI",
    "indirim_yuzdesi": "INDIRIM_YUZDESI",
    "aktivite_tipi": "AKTIVITE_TIPI",
    "tahmin_edilme_yontemi": "TAHMIN_EDILME_YONTEMI",
    "datanin_etiketlendigi_algorit": "DATANIN_ETIKETLENDIGI_ALGORIT",
    "tahmin_sapmasi": "TAHMIN_SAPMASI",
    "tl_unit": "TL_UNIT",
    "kg_unit": "KG_UNIT",
    "kg": "KG",
    "tl": "TL",
    "tahmin_koli_alt_sinir": "TAHMIN_KOLI_ALT_SINIR",
    "tahmin_koli_ust_sinir": "TAHMIN_KOLI_UST_SINIR",
    "aktivite_etkisi": "AKTIVITE_ETKISI"

}

In [268]:
col_name_dwh_order = [i for i in col_name_dwh.values()]

In [269]:
all_final_preds6.tahmin_koli.fillna(0, inplace=True)

In [270]:
all_final_preds6["mape_regulation"] = all_final_preds6["mape"].apply(lambda x: 100 if x>100 else x)

In [271]:
all_final_preds6["tahmin_koli_alt_sinir"] = all_final_preds6["tahmin_koli"] - (all_final_preds6["tahmin_koli"]*(all_final_preds6["mape"] / 100))
all_final_preds6["tahmin_koli_ust_sinir"] = all_final_preds6["tahmin_koli"] + (all_final_preds6["tahmin_koli"]*(all_final_preds6["mape"] / 100))

In [272]:
fiyat_unique_list = fiyat_unique[["Kanal", "en_guncel_kod"]].drop_duplicates(ignore_index=True)

In [273]:
def fiyat_guncel_calculate(fiyat_unique, fiyat_unique_list, idx):
    print("Basladı -->", idx)
    tmp = fiyat_unique[(fiyat_unique["Kanal"] == fiyat_unique_list.loc[idx, "Kanal"]) & 
                       (fiyat_unique["en_guncel_kod"] == fiyat_unique_list.loc[idx, "en_guncel_kod"])]
    tmp = tmp[tmp["date"] == tmp["date"].max()]
    print("Bitti -->", idx)
    return tmp

In [274]:
fiyat_guncel = []
if __name__ == "__main__" or __name__ == "__parents_main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    func = partial(fiyat_guncel_calculate, fiyat_unique, fiyat_unique_list)
    loop = list(fiyat_unique_list.index)
    with mp.Pool(available_cpu) as p:
        fiyat_guncel.append(p.map(func, loop))

In [275]:
fiyat_guncel = pd.concat(fiyat_guncel[0], ignore_index=True)

In [276]:
fiyat_guncel_btt = fiyat_guncel[fiyat_guncel["Kanal"] == "horizon"]
fiyat_guncel_btt["Kanal"] = "btt"
fiyat_guncel = pd.concat([fiyat_guncel, fiyat_guncel_btt], ignore_index=True, axis=0)
fiyat_guncel.rename(columns={"fiyat": "tl_unit"}, inplace=True)

In [277]:
all_final_preds6.shape

(209924, 18)

In [278]:
all_final_preds6 = all_final_preds6.merge(fiyat_guncel[["Kanal", "en_guncel_kod", "tl_unit"]], how="left")

In [279]:
all_final_preds6.shape

(209924, 19)

In [280]:
all_final_preds6["tl"] = all_final_preds6["tahmin_koli"] * all_final_preds6["tl_unit"]

In [281]:
all_final_preds6.drop("mape_regulation", axis=1, inplace=True)

In [282]:
koli_birim_agirlik.rename(columns={"kanal": "Kanal", "kg": "kg_unit"}, inplace=True)
koli_birim_agirlik = koli_birim_agirlik[["Kanal", "en_guncel_kod", "kg_unit"]]

In [283]:
all_final_preds6.shape

(209924, 19)

In [284]:
all_final_preds6 = all_final_preds6.merge(koli_birim_agirlik, how="left")
all_final_preds6.kg_unit.fillna(0, inplace=True)
all_final_preds6["kg"] = all_final_preds6["kg_unit"] * all_final_preds6["tahmin_koli"]

In [285]:
all_final_preds6.shape

(209924, 21)

In [286]:
koli_ici_adet_unique = koli_ici_adet_all[~(koli_ici_adet_all["grup_adi"].isin(['A101', 'BİM', 'Diğer_Pasifik', 'ŞOK', 'MİGROS']))][["grup_adi", "en_guncel_kod"]].drop_duplicates(ignore_index=True)

In [287]:
def koli_ici_adet_calculate(koli_ici_adet_all, koli_ici_adet_unique, idx):
    print("Basladı -->", idx)
    tmp = koli_ici_adet_all[(koli_ici_adet_all["grup_adi"] == koli_ici_adet_unique.loc[idx, "grup_adi"]) & 
                            (koli_ici_adet_all["en_guncel_kod"] == koli_ici_adet_unique.loc[idx, "en_guncel_kod"])]
    tmp = tmp[tmp["tarih"] == tmp["tarih"].max()]
    print("Bitti -->", idx)
    return tmp

In [288]:
koli_ici_adet_all2 = []
if __name__ == "__main__" or __name__ == "__parents_main__":
    mp.freeze_support()
    available_cpu = mp.cpu_count() - 1
    func = partial(koli_ici_adet_calculate, koli_ici_adet_all, koli_ici_adet_unique)
    loop = list(koli_ici_adet_unique.index)
    with mp.Pool(available_cpu) as p:
        koli_ici_adet_all2.append(p.map(func, loop))

In [289]:
koli_ici_adet_all2 = pd.concat(koli_ici_adet_all2[0], ignore_index=True)

In [290]:
koli_ici_adet_all3 = pd.concat([koli_ici_adet_all2, koli_ici_adet_all[koli_ici_adet_all.grup_adi.isin(["A101", "BİM", "Diğer_Pasifik", "MİGROS", "ŞOK"])]], ignore_index=True, axis=0)
koli_ici_adet_all3.drop("tarih", axis=1, inplace=True)

In [291]:
koli_ici_adet_all3.drop_duplicates(subset=["grup_adi", "en_guncel_kod"], ignore_index=True, inplace=True)

In [292]:
all_final_preds6.shape

(209924, 21)

In [293]:
all_final_preds6 = all_final_preds6.merge(koli_ici_adet_all3, how="left")

In [294]:
all_final_preds6.shape

(209924, 22)

In [295]:
all_final_preds6_backup = all_final_preds6.copy()

In [296]:
all_final_preds6["tahmin_adet"] = all_final_preds6["koli_ici_adet"] * all_final_preds6["tahmin_koli"]
all_final_preds6["tahmin_ust_sinir"] = all_final_preds6["koli_ici_adet"] * all_final_preds6["tahmin_koli_ust_sinir"]
all_final_preds6["tahmin_alt_sinir"] = all_final_preds6["koli_ici_adet"] * all_final_preds6["tahmin_koli_alt_sinir"]

In [297]:
all_final_preds6.shape

(209924, 25)

In [298]:
all_final_preds6 = all_final_preds6.merge(urun_isimleri, how="left")

In [299]:
all_final_preds6.shape

(209924, 26)

In [300]:
all_final_preds6.en_guncel_kod = all_final_preds6.en_guncel_kod.astype("int64")

In [301]:
all_final_preds6["datanin_etiketlendigi_algorit"] = all_final_preds6["tahmin_edilme_yontemi"]

In [302]:
set([i for i in col_name_dwh.keys()]) - set(all_final_preds6.columns)

{'aktivite_etkisi'}

In [303]:
set(all_final_preds6.columns) - set([i for i in col_name_dwh.keys()])

set()

In [304]:
all_final_preds6__backup = all_final_preds6.copy()

In [305]:
all_final_preds6.shape

(209924, 27)

In [306]:
all_final_preds6.drop_duplicates(subset=["en_guncel_kod", "grup_adi", "date", "aktivite_tipi", "indirim_yuzdesi"], ignore_index=True, inplace=True)

In [307]:
all_final_preds6.shape

(209924, 27)

In [308]:
all_final_preds7 = all_final_preds6.copy()

In [309]:
pasifik_akt_list = list(itertools.product(all_final_preds7.aktivite_tipi.unique(), all_final_preds7.indirim_yuzdesi.unique()))
pasifik_akt_list = [str(i[0])+"_"+str(i[1]) for i in pasifik_akt_list]

In [310]:
horizon_akt_list = list(itertools.product(all_final_preds7[all_final_preds7["Kanal"] != "pasifik"].aktivite_tipi.unique(), all_final_preds7.indirim_yuzdesi.unique()))
horizon_akt_list = [str(i[0])+"_"+str(i[1]) for i in horizon_akt_list]

In [311]:
all_final_preds8 = all_final_preds7[~((all_final_preds7["Kanal"].isin(["horizon", "btt"])) & (all_final_preds7["aktivite_tipi"] != "Yok"))].reset_index(drop=True)

In [312]:
all_final_preds8["akt_ind"] = all_final_preds8["aktivite_tipi"].astype(str) + "_" + all_final_preds8["indirim_yuzdesi"].astype(str)
all_final_preds8["date"] = pd.to_datetime(all_final_preds8["date"])
grup_sku_unique["grup_sku"] = grup_sku_unique["grup_adi"].astype(str) + "_" + grup_sku_unique["en_guncel_kod"].astype(str)
liseli_haric_grup_list = list(set(grup_sku_unique["grup_sku"]) - set(listeli_olmayan_sku["grup_sku"]))
listeli_haric_grup_sku = grup_sku_unique[grup_sku_unique["grup_sku"].isin(liseli_haric_grup_list)]

# Eğer t+2 haricindeki aylar için kombinasyonlar dönmemiş ise aşağıdaki kısımları çalıştır.
---

In [313]:
all_final_preds9 = all_final_preds8.copy()

In [314]:
all_final_preds9.to_csv("tum_grup_kat_oncesi.csv", index=False)

In [315]:
try:
    all_final_preds9.drop("akt_ind", axis=1, inplace=True)
except:
    pass

In [316]:
all_final_preds9.shape

(209924, 27)

In [317]:
all_final_preds9.drop_duplicates(ignore_index=True, inplace=True)

In [318]:
all_final_preds9.shape

(209924, 27)

---
# Tüm Grup, Tüm Ana Kategori, Tüm Kategori ve Tüm Marka groupby
---

In [319]:
all_final_preds9["from"] = "row"

In [320]:
kanal_all = all_final_preds9.groupby(["date", "Kanal", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
kanal_all["grup_adi"] = "TÜM GRUPLAR"
kanal_all["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
kanal_all["kategori_adi"] = "TÜM KATEGORİLER"
kanal_all["marka_adi"] = "TÜM MARKALAR"
kanal_all["en_guncel_kod"] = "TÜM SKULAR"
kanal_all["urun_adi"] = "TÜM ÜRÜNLER"
kanal_all["tahmin_edilme_yontemi"] = "N/A"
kanal_all["datanin_etiketlendigi_algorit"] = "N/A"
#kanal_all["aktivite_etkisi"] = "Var"

In [321]:
kanal_all["from"] = "kanal_all"

In [322]:
grup_all = all_final_preds9.groupby(["date", "Kanal", "grup_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
grup_all["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
grup_all["kategori_adi"] = "TÜM KATEGORİLER"
grup_all["marka_adi"] = "TÜM MARKALAR"
grup_all["en_guncel_kod"] = "TÜM SKULAR"
grup_all["urun_adi"] = "TÜM ÜRÜNLER"
grup_all["tahmin_edilme_yontemi"] = "N/A"
grup_all["datanin_etiketlendigi_algorit"] = "N/A"
#grup_all["aktivite_etkisi"] = "Var"

In [323]:
grup_all["from"] = "grup_all"

In [324]:
ana_kategori_all = all_final_preds9.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
ana_kategori_all["kategori_adi"] = "TÜM KATEGORİLER"
ana_kategori_all["marka_adi"] = "TÜM MARKALAR"
ana_kategori_all["en_guncel_kod"] = "TÜM SKULAR"
ana_kategori_all["urun_adi"] = "TÜM ÜRÜNLER"
ana_kategori_all["tahmin_edilme_yontemi"] = "N/A"
ana_kategori_all["datanin_etiketlendigi_algorit"] = "N/A"
#ana_kategori_all["aktivite_etkisi"] = "Var"

In [325]:
ana_kategori_all["from"] = "ana_kategori_all"

In [326]:
kategori_all = all_final_preds9.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", 
                                         "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()

kategori_all["marka_adi"] = "TÜM MARKALAR"
kategori_all["en_guncel_kod"] = "TÜM SKULAR"
kategori_all["urun_adi"] = "TÜM ÜRÜNLER"
kategori_all["tahmin_edilme_yontemi"] = "N/A"
kategori_all["datanin_etiketlendigi_algorit"] = "N/A"
#kategori_all["aktivite_etkisi"] = "Var"

In [327]:
kategori_all["from"] = "kategori_all"

In [328]:
marka_all = all_final_preds9.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", 
                                      "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()

marka_all["en_guncel_kod"] = "TÜM SKULAR"
marka_all["urun_adi"] = "TÜM ÜRÜNLER"
marka_all["tahmin_edilme_yontemi"] = "N/A"
marka_all["datanin_etiketlendigi_algorit"] = "N/A"
#marka_all["aktivite_etkisi"] = "Var"

In [329]:
marka_all["from"] = "marka_all"

In [330]:
# Hiyerarşik detay kırılımları ile

In [331]:
grup_detail = []
for idx in all_final_preds9.grup_adi.unique():
    tmp = all_final_preds9[all_final_preds9["grup_adi"] == idx]
    tmp = tmp.groupby(["date", "Kanal", "grup_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
    tmp["ana_kategori_adi"] = "TÜM ANA KATEGORİLER"
    tmp["kategori_adi"] = "TÜM KATEGORİLER"
    tmp["marka_adi"] = "TÜM MARKALAR"
    tmp["en_guncel_kod"] = "TÜM SKULAR"
    tmp["urun_adi"] = "TÜM ÜRÜNLER"
    tmp["tahmin_edilme_yontemi"] = "N/A"
    tmp["datanin_etiketlendigi_algorit"] = "N/A"
#    tmp["aktivite_etkisi"] = "Var"
    grup_detail.append(tmp)
grup_detail = pd.concat(grup_detail, ignore_index=True)

In [332]:
grup_detail["from"] = "grup_detail"

In [333]:
ana_kategori_detail = []
for idx in all_final_preds9.ana_kategori_adi.unique():
    tmp = all_final_preds9[all_final_preds9["ana_kategori_adi"] == idx]
    kontrol = tmp.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).agg({"kategori_adi": "nunique"}).reset_index().kategori_adi.unique()[0]
    tmp = tmp.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
    if kontrol == 1:
        pass
    else:
        tmp["kategori_adi"] = "TÜM KATEGORİLER"
        tmp["marka_adi"] = "TÜM MARKALAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
#        tmp["aktivite_etkisi"] = "Var"
        ana_kategori_detail.append(tmp)
ana_kategori_detail = pd.concat(ana_kategori_detail, ignore_index=True)

In [334]:
ana_kategori_detail["from"] = "ana_kategori_detail"

In [335]:
liste = all_final_preds9.drop_duplicates(subset=["Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi"])[["Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi"]].reset_index(drop=True)
kategori_detail = []
for idx in liste.index:
    tmp = all_final_preds9[(all_final_preds9["Kanal"] == liste["Kanal"][idx]) & 
                           (all_final_preds9["grup_adi"] == liste["grup_adi"][idx]) & 
                           (all_final_preds9["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                           (all_final_preds9["kategori_adi"] == liste["kategori_adi"][idx])]
    kontrol = len(tmp["marka_adi"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["marka_adi"] = "TÜM MARKALAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
#        tmp["aktivite_etkisi"] = "Var"
        kategori_detail.append(tmp)
kategori_detail = pd.concat(kategori_detail, ignore_index=True)

In [336]:
kategori_detail["from"] = "kategori_detail"

In [337]:
liste = all_final_preds9.drop_duplicates(subset=["Kanal", "grup_adi", "ana_kategori_adi", 
                                                 "kategori_adi", "marka_adi"])[["Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi"]].reset_index(drop=True)
marka_detail = []
for idx in liste.index:
    tmp = all_final_preds9[(all_final_preds9["Kanal"] == liste["Kanal"][idx]) & 
                           (all_final_preds9["grup_adi"] == liste["grup_adi"][idx]) & 
                           (all_final_preds9["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                           (all_final_preds9["kategori_adi"] == liste["kategori_adi"][idx]) & 
                           (all_final_preds9["marka_adi"] == liste["marka_adi"][idx])]
    kontrol = len(tmp["en_guncel_kod"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["date", "Kanal", "grup_adi", "ana_kategori_adi", "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
#        tmp["aktivite_etkisi"] = "Var"
        marka_detail.append(tmp)
marka_detail = pd.concat(marka_detail, ignore_index=True)

In [338]:
marka_detail["from"] = "marka_detail"

In [339]:
tum_gruplar_ana_kategori = all_final_preds9.groupby(["date", "Kanal", "ana_kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_ana_kategori["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_ana_kategori["kategori_adi"] = "TÜM KATEGORİLER"
tum_gruplar_ana_kategori["marka_adi"] = "TÜM MARKALAR"
tum_gruplar_ana_kategori["en_guncel_kod"] = "TÜM SKULAR"
tum_gruplar_ana_kategori["urun_adi"] = "TÜM ÜRÜNLER"
tum_gruplar_ana_kategori["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_ana_kategori["datanin_etiketlendigi_algorit"] = "N/A"
#tum_gruplar_ana_kategori["aktivite_etkisi"] = "Var"

In [340]:
tum_gruplar_ana_kategori["from"] = "tum_gruplar_ana_kategori"

In [341]:
tum_gruplar_kategori = all_final_preds9.groupby(["date", "Kanal", "ana_kategori_adi", "kategori_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_kategori["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_kategori["marka_adi"] = "TÜM MARKALAR"
tum_gruplar_kategori["en_guncel_kod"] = "TÜM SKULAR"
tum_gruplar_kategori["urun_adi"] = "TÜM ÜRÜNLER"
tum_gruplar_kategori["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_kategori["datanin_etiketlendigi_algorit"] = "N/A"
#tum_gruplar_kategori["aktivite_etkisi"] = "Var"

In [342]:
tum_gruplar_kategori["from"] = "tum_gruplar_kategori"

In [343]:
liste = all_final_preds9.drop_duplicates(subset=["Kanal", "ana_kategori_adi", 
                                                 "kategori_adi", "marka_adi"])[["Kanal", "ana_kategori_adi", "kategori_adi", "marka_adi"]].reset_index(drop=True)

In [344]:
tum_gruplar_marka = []
for idx in liste.index:
    tmp = all_final_preds9[(all_final_preds9["Kanal"] == liste["Kanal"][idx]) & 
                           (all_final_preds9["ana_kategori_adi"] == liste["ana_kategori_adi"][idx]) & 
                           (all_final_preds9["kategori_adi"] == liste["kategori_adi"][idx]) & 
                           (all_final_preds9["marka_adi"] == liste["marka_adi"][idx])]
    kontrol = len(tmp["en_guncel_kod"].unique())
    if kontrol == 1:
        pass
    else:
        tmp = tmp.groupby(["date", "Kanal", "ana_kategori_adi", "kategori_adi", "marka_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
        tmp["grup_adi"] = "TÜM GRUPLAR"
        tmp["en_guncel_kod"] = "TÜM SKULAR"
        tmp["urun_adi"] = "TÜM ÜRÜNLER"
        tmp["tahmin_edilme_yontemi"] = "N/A"
        tmp["datanin_etiketlendigi_algorit"] = "N/A"
#        tmp["aktivite_etkisi"] = "Var"
        tum_gruplar_marka.append(tmp)
tum_gruplar_marka = pd.concat(tum_gruplar_marka, ignore_index=True)

In [345]:
tum_gruplar_marka["from"] = "tum_gruplar_marka"

In [346]:
tum_gruplar_en_guncel_kod = all_final_preds9.groupby(["date", "Kanal", "ana_kategori_adi", "kategori_adi", "marka_adi", "en_guncel_kod", "urun_adi", "indirim_yuzdesi", "aktivite_tipi"]).sum().reset_index()
tum_gruplar_en_guncel_kod["grup_adi"] = "TÜM GRUPLAR"
tum_gruplar_en_guncel_kod["tahmin_edilme_yontemi"] = "N/A"
tum_gruplar_en_guncel_kod["datanin_etiketlendigi_algorit"] = "N/A"
#tum_gruplar_en_guncel_kod["aktivite_etkisi"] = "Var"

In [347]:
tum_gruplar_en_guncel_kod["from"] = "tum_gruplar_en_guncel_kod"

In [348]:
chk = all_final_preds9.groupby(["grup_adi", "en_guncel_kod", "date", "indirim_yuzdesi", "aktivite_tipi"]).count().reset_index()
kntrl = chk[chk["Kanal"] > 1].drop_duplicates(subset=["grup_adi", "en_guncel_kod", "date", "indirim_yuzdesi", "aktivite_tipi"])

# Boş gelmesi doğru olduğuna işaret ediyor
---

In [349]:
kntrl

,grup_adi,en_guncel_kod,date,indirim_yuzdesi,aktivite_tipi,tahmin_koli,mape,tahmin_edilme_yontemi,gerceklesen_satis_adedi,gerceklesen_koli,tahmin_sapmasi,Kanal,ana_kategori_adi,kategori_adi,marka_adi,tahmin_koli_alt_sinir,tahmin_koli_ust_sinir,tl_unit,tl,kg_unit,kg,koli_ici_adet,tahmin_adet,tahmin_ust_sinir,tahmin_alt_sinir,urun_adi,datanin_etiketlendigi_algorit,from


In [350]:
kanal_all = kanal_all[all_final_preds9.columns.to_list()]
grup_all = grup_all[all_final_preds9.columns.to_list()]
ana_kategori_all = ana_kategori_all[all_final_preds9.columns.to_list()]
kategori_all = kategori_all[all_final_preds9.columns.to_list()]
marka_all = marka_all[all_final_preds9.columns.to_list()]
grup_detail = grup_detail[all_final_preds9.columns.to_list()]
ana_kategori_detail = ana_kategori_detail[all_final_preds9.columns.to_list()]
kategori_detail = kategori_detail[all_final_preds9.columns.to_list()]
marka_detail = marka_detail[all_final_preds9.columns.to_list()]
tum_gruplar_ana_kategori = tum_gruplar_ana_kategori[all_final_preds9.columns.to_list()]
tum_gruplar_kategori = tum_gruplar_kategori[all_final_preds9.columns.to_list()]
tum_gruplar_marka = tum_gruplar_marka[all_final_preds9.columns.to_list()]
tum_gruplar_en_guncel_kod = tum_gruplar_en_guncel_kod[all_final_preds9.columns.to_list()]

In [351]:
all_final_preds10 = pd.concat([all_final_preds9,
                              kanal_all,
                              grup_all,
                              ana_kategori_all,
                              kategori_all,
                              marka_all,
                              grup_detail,
                              ana_kategori_detail,
                              kategori_detail,
                              marka_detail,
                              tum_gruplar_ana_kategori,
                              tum_gruplar_kategori,
                              tum_gruplar_marka,
                              tum_gruplar_en_guncel_kod], axis=0, ignore_index=True)

In [358]:
all_final_preds10.drop_duplicates(ignore_index=True, inplace=True)

In [360]:
all_final_preds10.columns = [i[:-2] if len(i) > 30 else i for i in all_final_preds10.columns]
all_final_preds10["en_guncel_kod"] = all_final_preds10["en_guncel_kod"].apply(lambda x: "TÜM SKULAR" if x == "TÜM SKU'LAR" else x)

In [361]:
grup_and_sku_list = all_final_preds10.drop_duplicates(subset=["grup_adi", "en_guncel_kod"], ignore_index=True)[["grup_adi", "en_guncel_kod"]]
grup_and_sku_list = grup_and_sku_list[grup_and_sku_list["en_guncel_kod"] != "TÜM SKULAR"].reset_index(drop=True)

In [362]:
all_final_preds10["aktivite_tipi"] = all_final_preds10["aktivite_tipi"].apply(lambda x: "In-out" if x=="İn&out" else x)

In [363]:
all_final_preds10["Kanal"] = all_final_preds10["Kanal"].str.replace("'", "")
all_final_preds10["grup_adi"] = all_final_preds10["grup_adi"].str.replace("'", "")
all_final_preds10["ana_kategori_adi"] = all_final_preds10["ana_kategori_adi"].str.replace("'", "")
all_final_preds10["kategori_adi"] = all_final_preds10["kategori_adi"].str.replace("'", "")
all_final_preds10["marka_adi"] = all_final_preds10["marka_adi"].str.replace("'", "")
all_final_preds10["urun_adi"] = all_final_preds10["urun_adi"].str.replace("'", "")

In [364]:
df_all_v6["indirim__bins"] = df_all_v6["indirim__"].apply(lambda x: 0 if x <= 0 else 
                                                          (1 if x <= 0.01 else 
                                                           (2 if x <= 0.02 else 
                                                            (3 if x <= 0.03 else 
                                                             (4 if x <= 0.04 else 
                                                              (5 if x <= 0.05 else 
                                                               (6 if x <= 0.06 else 
                                                                (7 if x <= 0.07 else 
                                                                 (8 if x <= 0.08 else 
                                                                  (9 if x <= 0.09 else 
                                                                   (10 if x <= 0.10 else 
                                                                    (12.5 if x <= 0.15 else 15))))))))))))

In [365]:
all_final_preds10.shape
#bir önceki iteration'da 443460 satır idi neden?

(463320, 28)

In [366]:
all_final_preds10["aktivite_etkisi"] = np.nan
for idx in grup_and_sku_list.index:
    akt = df_all[(df_all["grup_adi"] == grup_and_sku_list["grup_adi"][idx]) & 
                 (df_all["en_guncel_kod"] == grup_and_sku_list["en_guncel_kod"][idx])]["aktivite_tipi"].unique()
    
    ind = df_all_v6[(df_all_v6["grup_adi"] == grup_and_sku_list["grup_adi"][idx]) & 
                    (df_all_v6["en_guncel_kod"] == grup_and_sku_list["en_guncel_kod"][idx])]["indirim__bins"].unique()
    
    indexes = all_final_preds10[(all_final_preds10["grup_adi"] == grup_and_sku_list["grup_adi"][idx]) & 
                                (all_final_preds10["en_guncel_kod"] == grup_and_sku_list["en_guncel_kod"][idx])].index
    
    if (len(akt) == 1 and len(ind) == 1) and (akt[0] == 0 and ind[0] == 0):
        all_final_preds10.loc[list(indexes), "aktivite_etkisi"] = "Yok"
    else:
        all_final_preds10.loc[list(indexes), "aktivite_etkisi"] = "Var"
all_final_preds10.aktivite_etkisi.fillna("Var", inplace=True)

In [367]:
all_final_preds10.shape #(443460, 29)

(463320, 29)

In [368]:
try:
    all_final_preds10.drop(columns="from", axis=1, inplace=True)
except:
    pass

In [369]:
all_final_preds10["en_guncel_kod"] = all_final_preds10["en_guncel_kod"].apply(lambda x: int(x) if "T" not in str(x) else x)

In [370]:
all_final_preds10.head()

,en_guncel_kod,grup_adi,tahmin_koli,mape,tahmin_edilme_yontemi,date,aktivite_tipi,indirim_yuzdesi,gerceklesen_satis_adedi,gerceklesen_koli,tahmin_sapmasi,Kanal,ana_kategori_adi,kategori_adi,marka_adi,tahmin_koli_alt_sinir,tahmin_koli_ust_sinir,tl_unit,tl,kg_unit,kg,koli_ici_adet,tahmin_adet,tahmin_ust_sinir,tahmin_alt_sinir,urun_adi,datanin_etiketlendigi_algorit,aktivite_etkisi
0,107701,A101,29868.906573,31.852574,DIAG1_UCM1,2022-03-01,Kasiyer,%3 - %4,0,0,0,pasifik,ATIŞTIRMALIK,BİSKÜVİ,İKRAM,20354.890945,39382.922200,88.92,2.655943e+06,1.0,29868.906573,12.0,358426.878873,472595.066401,244258.691344,İKRAM KRE.BİSK.ÇİK.3x84Gx12,DIAG1_UCM1,Var
1,107701,A101,21461.742583,31.852574,DIAG1_UCM1,2022-04-01,Çoklu Alım,%8 - %9,0,0,0,pasifik,ATIŞTIRMALIK,BİSKÜVİ,İKRAM,14625.625103,28297.860063,88.92,1.908378e+06,1.0,21461.742583,12.0,257540.910999,339574.320757,175507.501240,İKRAM KRE.BİSK.ÇİK.3x84Gx12,DIAG1_UCM1,Var
2,107701,A101,23793.070111,31.852574,DIAG1_UCM1,2022-03-01,Mağaza içi/Dağılım,%4 - %5,0,0,0,pasifik,ATIŞTIRMALIK,BİSKÜVİ,İKRAM,16214.364801,31371.775422,88.92,2.115680e+06,1.0,23793.070111,12.0,285516.841337,376461.305064,194572.377610,İKRAM KRE.BİSK.ÇİK.3x84Gx12,DIAG1_UCM1,Var
3,107701,A101,23556.143584,31.852574,DIAG1_UCM1,2022-06-01,Kasiyer,0,0,0,0,pasifik,ATIŞTIRMALIK,BİSKÜVİ,İKRAM,16052.905471,31059.381696,88.92,2.094612e+06,1.0,23556.143584,12.0,282673.723005,372712.580356,192634.865654,İKRAM KRE.BİSK.ÇİK.3x84Gx12,DIAG1_UCM1,Var
4,107701,A101,34247.099222,31.852574,DIAG1_UCM1,2022-03-01,Kasiyer,%6 - %7,0,0,0,pasifik,ATIŞTIRMALIK,BİSKÜVİ,İKRAM,23338.516532,45155.681911,88.92,3.045252e+06,1.0,34247.099222,12.0,410965.190658,541868.182929,280062.198387,İKRAM KRE.BİSK.ÇİK.3x84Gx12,DIAG1_UCM1,Var


In [371]:
all_final_preds10.rename(columns=col_name_dwh, inplace=True)
all_final_preds10 = all_final_preds10[col_name_dwh_order]

kanal_order = {'pasifik': 'PASİFİK', 
               'btt': 'BTT', 
               'horizon': 'HORİZON'}

In [372]:
all_final_preds10.KANAL.unique()

array(['pasifik', 'btt', 'horizon'], dtype=object)

# Kanal isimlerini kanal_order dictionary'sindeki haline getiriyorum ve indirim yüzdesinde integer varsa stringe çeviriyorum
---

In [373]:
all_final_preds10["KANAL"] = all_final_preds10["KANAL"].map(kanal_order)
all_final_preds10["INDIRIM_YUZDESI"] = all_final_preds10["INDIRIM_YUZDESI"].apply(lambda x: str(x))

# Ne olur ne olmaz diye drop duplicates yapıyorum tekrarlayan satırlar olmasın diye
---

In [374]:
all_final_preds10.drop_duplicates(ignore_index=True, inplace=True)
all_final_preds10_backup = all_final_preds10.copy()

# kg hesabı yaparken tahmin koli ile bir kolinin ağırlığını çarpıyorum
---

In [376]:
all_final_preds10["KG"] = all_final_preds10["TAHMIN_KOLI"] * all_final_preds10["KG_UNIT"]

# tahminin kilo karşılığını getirirken virgülden sonra 3 hane, tl için virgülden sonra 2 hane bırakıyorum. tahminleri de yukarıya yuvarlıyorum. Mesela 155.56 ise 156 oluyor
---

In [377]:
all_final_preds10["KG"] = np.round(all_final_preds10["KG"], 3)
all_final_preds10["TL"] = np.round(all_final_preds10["TL"], 2)
all_final_preds10["TAHMIN_KOLI"] = np.ceil(all_final_preds10["TAHMIN_KOLI"])
all_final_preds10["TAHMIN_KOLI_ALT_SINIR"] = np.ceil(all_final_preds10["TAHMIN_KOLI_ALT_SINIR"])
all_final_preds10["TAHMIN_KOLI_UST_SINIR"] = np.ceil(all_final_preds10["TAHMIN_KOLI_UST_SINIR"])
all_final_preds10["TARIH"] = all_final_preds10["TARIH"].astype(str)

# Horizon için de aktivite girilmişse çıkartmak amacıyla yazdım aşağıdakini
---

In [378]:
all_final_preds11 = all_final_preds10[~((all_final_preds10["KANAL"].isin(["HORİZON", "BTT"])) & (all_final_preds10["AKTIVITE_TIPI"] != "Yok"))].reset_index(drop=True)

# shape'leri kontrol etmek için yazdım
---

In [389]:
try: 
    conn.userinfo()
except: 
    print("SWAT bağlantısı koptu. Bağlantı tekrar kuruluyor.")
    access_ = True
    while access_:
        try:
            conn = swat.CAS('yhtrcl-sasccnt1.yildiz.domain', 5570, username=params_["login_info"]["username"], password=params_["login_info"]["password"])
            access_ = False
            print("Bağlantı kuruldu!")
        except:
            print("Beklenmedik bir hata oluştu. Bağlantı tekrar kuruluyor...")
            continue
#ds_table = conn.CASTable("DEMAND_SENSING_RESULTS", caslib=params_["caslib_info"]["caslib_name"], replace=True)
#ds_table.table.dropTable(quiet=True)
if conn.table.tableExists(name="DEMAND_SENSING_RESULTS",caslib=params_['caslib_info']['caslib_name']).exists:
    conn.table.dropTable(name="DEMAND_SENSING_RESULTS",caslib=params_['caslib_info']['caslib_name'])
conn.upload(data=all_final_preds11, 
            casout={'caslib':params_["caslib_info"]["caslib_name"], 'name':"DEMAND_SENSING_RESULTS", 'promote':True})

SWAT bağlantısı koptu. Bağlantı tekrar kuruluyor.
Bağlantı kuruldu!
NOTE: Cloud Analytic Services dropped table DEMAND_SENSING_RESULTS from caslib DSENS_P.
NOTE: Cloud Analytic Services made the uploaded file available as table DEMAND_SENSING_RESULTS in caslib DSENS_P.
NOTE: The table DEMAND_SENSING_RESULTS has been created in caslib DSENS_P from binary data uploaded to Cloud Analytic Services.


[caslib]

 'DSENS_P'

[tableName]

 'DEMAND_SENSING_RESULTS'

[casTable]

 CASTable('DEMAND_SENSING_RESULTS', caslib='DSENS_P')

+ Elapsed: 200s, user: 4.52s, sys: 1.69s, mem: 85.5mb

In [390]:
listeli_export = df_all_v6[["Kanal", "grup_adi", "en_guncel_kod", "portfoy", "durum"]]
listeli_export = listeli_export[listeli_export["Kanal"] == "pasifik"]
listeli_export.drop("Kanal", axis=1, inplace=True)
listeli_export.drop_duplicates(ignore_index=False, inplace=True)
listeli_export.to_excel("../data/listeli_urun_listesi_"+data_name_dict[params_["time_info_for_debugging"]["ay"]]+"_"+str(params_["time_info_for_debugging"]["yil"])+".xlsx", index=False)

In [391]:
chng_names = {"KANAL": "Kanal", 
              "GRUP_ADI": "Grup Adı", 
              "ANA_KATEGORI_ADI": "Ana Kategori",
              "KATEGORI_ADI": "Kategori", 
              "MARKA_ADI": "Marka", 
              "URUN_ADI": "Ürün Adı", 
              "EN_GUNCEL_KOD": "Ürün Kodu"}

all_possile_selections = all_final_preds11[["KANAL", "GRUP_ADI", "ANA_KATEGORI_ADI", "KATEGORI_ADI", "MARKA_ADI", "URUN_ADI", "EN_GUNCEL_KOD"]]
all_possile_selections.drop_duplicates(ignore_index=True, inplace=True)
all_possile_selections.rename(columns=chng_names, inplace=True)

In [392]:
all_possile_selections.to_excel("../data/__all_possible_selections.xlsx", index=False)

In [393]:
df_all.to_csv("../data/historic_data_for_graph.csv", index=False)
df_all_v6.to_csv("../data/historic_data_trend_for_graph.csv", index=False)

---
# Geçmiş ve güncel predictionların database'e yazılması
---

In [394]:
database_upload = all_final_preds11.copy()

In [395]:
database_upload["DATANIN_KESILDIGI_TARIH"] = datetime(params_["time_info_for_debugging"]["yil"], params_["time_info_for_debugging"]["ay"], 1)
database_upload["TAHMINLENEN_TARIH"] = t_2_date

In [397]:
df_sas_upload["datanin_kesildigi_tarih"] = datetime(params_["time_info_for_debugging"]["yil"], params_["time_info_for_debugging"]["ay"], 1)

In [482]:
if __name__ == '__main__':
    
#    create_connection(r"../data/db/"+params_["db_info"]["db_name"]+".sqlite") # creates new database if there is no db.
#    create_new_table(all_final_preds11, params_["db_info"]["db_name"], params_["db_info"]["current_results_table"]) # creates new table if there is no table.
#    upload_new_results(all_final_preds11, params_["db_info"]["db_name"], params_["db_info"]["current_results_table"]) # drop previous predictions and uploads new predictions for current period.
    
    update_table(database_upload, params_["db_info"]["db_name"], params_["db_info"]["history_results_table"]) # inserts to backup database.
#    update_table(df_sas_upload, params_["db_info"]["db_name"], params_["db_info"]["historic_data"]) # inserts to historic database.

New data is inserted into DS_HISTORY_RESULTS table.
New data is inserted into DS_HISTORY_DATA table.


In [483]:
conn = sqlite3.connect("../data/db/DS_DATABASE.sqlite")

In [485]:
a = pd.read_sql_query("SELECT * FROM DS_HISTORY_RESULTS", con=conn)
b = pd.read_sql_query("SELECT * FROM DS_HISTORY_DATA", con=conn)

In [499]:
conn.close()